In [ ]:
# Install required packages.
import os

#!pip install torch==1.7.0
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)


!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install git+https://github.com/rusty1s/pytorch_geometric.git

2.2.1+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 25.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/rusty1s/pytorch_geometric.git to /tmp/pip-req-build-7qehxh1q
  Running command git clone --filter=blob:none --quiet https://github.com/rusty1s/pytorch_geometric.git /tmp/pip-req-build-7qehxh1q
  Resolved https://github.com/rusty1s/pytorch_geometric.git to commit e9648df16dcb6dde0e09b5736b1b2da5d68db2ad
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import argparse
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from math import sqrt
from statistics import mean
import torch_geometric
from torch_geometric.datasets import TUDataset
import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch.nn import Linear, ReLU, Sequential
from sklearn import metrics
from scipy.spatial.distance import hamming
import statistics
import pandas
import csv
from time import perf_counter
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.loader import DataLoader
import torch_geometric.nn as gnn
from torch.autograd import graph
from typing import Any, Dict, Optional, Union
from IPython.core.display import deepcopy
from torch_geometric.nn import MessagePassing
import copy
from importlib import reload
import pickle
from sklearn.preprocessing import label_binarize
from tqdm import tqdm
from torch_geometric.data import Data, Batch, Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
nci1_dataset = TUDataset(root='data/TUDataset', name='NCI1')

Processing...
Done!


In [ ]:
Explainability_name = 'Run All Methods'
Task_name = 'Graph Classification'
checkpoint_directory_Classifier = "/content/drive/My Drive/Explainability Methods/" + str(Explainability_name) + " on " + str(Task_name) + "/Model/model_classifier.pt"
classifier_lr = 0.001
classifier_dropout = 0.1
classifier_weight_decay = 1e-6
classifier_bias = True
DataSet_name = "NCI1"

#File_Name = Model_name + " " + Explainability_name + " " + Task_name + " " + DataSet_name + " "

In [ ]:
df = pandas.read_csv("/content/drive/My Drive/Explainability Methods/Train and Test Indexes on Graph Classification/Experimental Results/train_test_indexes_NCI1.csv")

read_training_list_indexes__ = df['Train Indexes']
read_test_list_indexes__ = df['Test Indexes']
read_test_list_indexes__ = read_test_list_indexes__.dropna()
read_test_list_indexes = []
read_training_list_indexes = []
for element in read_test_list_indexes__:
    read_test_list_indexes.append(int(element))
for element in read_training_list_indexes__:
    read_training_list_indexes.append(int(element))


print(read_training_list_indexes)
print(read_test_list_indexes)

[1769, 2293, 1110, 3802, 1484, 3715, 3654, 2414, 1099, 48, 2863, 1024, 3580, 2363, 4084, 3557, 1975, 589, 807, 2228, 2346, 2743, 1425, 1883, 597, 727, 1357, 797, 3780, 2367, 1039, 1111, 1267, 3667, 608, 3539, 277, 3999, 3907, 3860, 2648, 3457, 702, 1417, 398, 3826, 2318, 1662, 162, 3046, 3817, 1302, 3963, 3400, 858, 2172, 462, 956, 1286, 976, 216, 4028, 3339, 3043, 3665, 3535, 550, 674, 1291, 3013, 3349, 3964, 2224, 1818, 2181, 1473, 96, 785, 3336, 1587, 2728, 1232, 1771, 1354, 3641, 4069, 2467, 3850, 2581, 3369, 2956, 3442, 2617, 3481, 2604, 3123, 291, 3548, 987, 2820, 3022, 1433, 1893, 3590, 1085, 1886, 2455, 2182, 214, 357, 3182, 1488, 3492, 1646, 3956, 3044, 1123, 1850, 3456, 2031, 3804, 3894, 3951, 3725, 2393, 1361, 59, 1197, 1486, 1126, 1912, 1787, 197, 3272, 1884, 4052, 2205, 3560, 53, 2977, 50, 2557, 1789, 2575, 3703, 2531, 3247, 535, 3622, 3403, 891, 449, 1896, 3923, 3613, 399, 1561, 1004, 1610, 121, 1822, 119, 699, 2849, 1377, 1276, 2256, 3282, 861, 2006, 497, 2194, 2725, 319

In [ ]:
#train_dataset, test_dataset = train_test_split(dataset, test_size=0.3, random_state=0, shuffle=True)
#print("Number of Training Graphs: ", len(train_dataset))
#print("Number of Test Graphs: ", len(test_dataset))

nci1_train_dataset = []
nci1_test_dataset = []
for index in read_training_list_indexes:
    nci1_train_dataset.append(nci1_dataset[index])
for index in read_test_list_indexes:
    nci1_test_dataset.append(nci1_dataset[index])


print(f'Number of training graphs: {len(nci1_train_dataset)}')
print(f'Number of test graphs: {len(nci1_test_dataset)}')

Number of training graphs: 3288
Number of test graphs: 822


In [ ]:
print(nci1_train_dataset[0])

Data(edge_index=[2, 54], x=[25, 37], y=[1])


In [ ]:
BATCH_SIZE = 64
nci1_train_dataloader = DataLoader(nci1_train_dataset, batch_size=BATCH_SIZE, shuffle=False) # important to be false
nci1_test_dataloader = DataLoader(nci1_test_dataset, batch_size=1, shuffle=False)

In [ ]:
print(nci1_train_dataloader.batch_size)
batch = next(iter(nci1_train_dataloader))
print(batch.y)
print(len(nci1_train_dataloader))

64
tensor([1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
        0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1,
        0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1])
52


In [ ]:
# from torch_geometric.datasets import FakeDataset

# num_graphs = 100
# avg_num_nodes = 20
# avg_degree = 15
# node_feat_dim = 13
# edge_feat_dim = 0
# num_classes = 3

# fake_dataset = FakeDataset(num_graphs=num_graphs, avg_num_nodes=avg_num_nodes, avg_degree=avg_degree, num_channels=node_feat_dim,
#                            edge_dim=edge_feat_dim, num_classes=num_classes,)
# Explainability_name = 'PGExplainer'
# Task_name = 'Graph Classification'
# classifier_bias = True
# DataSet_name = "Fake"
# BATCH_SIZE = 10
# classifier_lr = 0.001
# classifier_dropout = 0.1
# classifier_weight_decay = 1e-6

# train_ratio = 0.8

# fake_train_dataset = fake_dataset[:int(len(fake_dataset) * train_ratio)]
# fake_test_dataset = fake_dataset[len(fake_dataset) - int(len(fake_dataset) * train_ratio):]

# fake_train_dataloader = DataLoader(fake_train_dataset, batch_size=BATCH_SIZE, shuffle=False)
# fake_test_dataloader = DataLoader(fake_test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
    #   node_feat_size = self.input_dataset[0].x.size()[-1]
    #   if self.GNN_Model_name == "GCN_plus_GAP":
    #     return gcn_plus_gap_model.GCN_plus_GAP(model_name="GCN_plus_GAP", act_fun=act_fun, model_level="graph", input_dim=node_feat_size, hidden_dim=hidden_dim, output_dim=self.num_classes, num_hid_layers=3, Bias=Bias, Weight_Initializer=winit, dropout_rate=dropout_rate)
    #   elif self.GNN_Model_name == 'DGCNN':
    #     k = 17
    #     if self.dataset_name == 'NCI1' or self.dataset_name == 'ENZYMES':
    #       k = 32
    #     elif self.dataset_name == 'Graph-SST5':
    #       k = 19

    #     return dgcnn_model.DGCNN_Model(GNN_layers=[32, 32, 32, 13], num_classes=self.num_classes, mlp_act_fun="ReLu", dgcnn_act_fun="tanh", mlp_dropout_rate=dropout_rate, Weight_Initializer=winit, Bias=Bias, dgcnn_k=k, node_feat_size=node_feat_size, hid_channels=[128,128], conv1d_kernels=[2,5], ffn_layer_size=128, strides=[2,1])

    #   elif self.GNN_Model_name == 'DIFFPOOL':
    #     return diffpool_model.DIFFPOOL_Model(embedding_input_dim=node_feat_size, embedding_num_block_layers=1, embedding_hid_dim=64, new_feature_size=64, assignment_input_dim=node_feat_size,
    #                                   assignment_num_block_layers=1, assignment_hid_dim=64, max_number_of_nodes=256, prediction_hid_layers=[50], concat_neighborhood=False, num_classes=self.num_classes, Weight_Initializer=winit, Bias=Bias, dropout_rate=dropout_rate, normalize_graphsage=False, aggregation="mean", act_fun=act_fun, concat_diffpools_outputs=True, num_pooling=1, pooling="mean")
    #   elif self.GNN_Model_name == 'GIN':
    #     return gin_model.GIN_Model(num_mlp_layers=3, Bias=Bias, num_slp_layers=2, mlp_input_dim=node_feat_size, mlp_hid_dim=hidden_dim, mlp_output_dim=self.num_classes, mlp_act_fun=act_fun, dropout_rate=dropout_rate, Weight_Initializer=winit, joint_embeddings= None)

In [ ]:
# NCI1,DGCNN,4 * tanh(DGCNN) + 2 * relu(Conv1d) + Sortpooling + 2 * relu(Linear),kaiming_normal_,5,1000,0.0001,0.3,True,32,CrossEntropyLoss
# NCI1,DIFFPOOL,2 * DIFFPOOL + 2 * GRAPHSAGE + 3 * Linear,xavier_normal_,2,1000,0.0001,0.3,True,32,CrossEntropyLoss
# NCI1,GCN_plus_GAP,ReLu(GCN1)+ReLu(GCN2)+GAP+FFN,xavier_normal_,1,1000,0.0001,0.3,True,32,CrossEntropyLoss
# NCI1,GIN,kaiming_normal_,5,200,0.001,0.3,True,32,CrossEntropyLoss

In [ ]:
import sys
py_path = '/content/drive/MyDrive/Explainability Methods/Models/Script/'
sys.path.insert(0,py_path)

import GCN_plus_GAP as Graph_Network
GNN_Model = Graph_Network.GCN_plus_GAP(model_name='GCN_plus_GAP', model_level='graph', input_dim=37, hidden_dim=128, output_dim=2,
                                       num_hid_layers=3, Bias=classifier_bias, act_fun='ReLu', Weight_Initializer=1,
                                       dropout_rate=0.3)

# import DGCNN as dgcnn_model
# GNN_Model = dgcnn_model.DGCNN_Model(GNN_layers=[32, 32, 32, 7], num_classes=2, node_feat_size=7, mlp_act_fun='ReLu',
#                                    dgcnn_act_fun='tanh', mlp_dropout_rate=0.5, Weight_Initializer=3, Bias=False, dgcnn_k=17,
#                                    hid_channels=[16,32], conv1d_kernels=[2,5], ffn_layer_size=128, strides=[2,1])

# import DIFFPOOL as diffpool_model
# GNN_Model = diffpool_model.DIFFPOOL_Model(embedding_input_dim=7, embedding_num_block_layers=1, embedding_hid_dim=64,
#                                          new_feature_size=64, assignment_input_dim=7, assignment_num_block_layers=1,
#                                          assignment_hid_dim=64, max_number_of_nodes=256, prediction_hid_layers=[50],
#                                          concat_neighborhood=False, num_classes=2, Weight_Initializer=1, Bias=classifier_bias,
#                                          dropout_rate=0, normalize_graphsage=False, aggregation="mean", act_fun="ReLu",
#                                          concat_diffpools_outputs=True, num_pooling=1, pooling="mean")

# import GIN as gin_model
# GNN_Model = gin_model.GIN_Model(num_mlp_layers=4, mlp_input_dim=37, mlp_hid_dim=37, mlp_output_dim=2, num_slp_layers=2,
#                                 Bias=classifier_bias, mlp_act_fun="ReLu", dropout_rate=classifier_dropout, Weight_Initializer=1,
#                                 joint_embeddings=False)



Model_name = GNN_Model.__class__.__name__

GCN_plus_GAP Input_Dimension: 37
GCN_plus_GAP Hidden_Dimension: 128
GCN_plus_GAP Output_Dimension: 2
GCN_plus_GAP Number_of_Hidden_Layers: 3
ReLu is Selected.


In [ ]:
print(GNN_Model)

GCN_plus_GAP(
  (GConvs): ModuleList(
    (0): GCNConv(37, 128)
    (1-2): 2 x GCNConv(128, 128)
  )
  (dropout): Dropout(p=0.3, inplace=False)
  (readout): GlobalMeanPool()
  (ffn): Linear(in_features=128, out_features=2, bias=True)
)


In [ ]:
GNN_Model_state_dict = torch.load("/content/drive/MyDrive/Explainability Methods/Run All Methods on Graph Classification/Model/GCN_plus_GAP_model_classifier_NCI1.pt")

In [ ]:
GNN_Model.load_state_dict(GNN_Model_state_dict['model_state_dict'])

<All keys matched successfully>

In [ ]:
GNN_Model_Optimizer = torch.optim.Adam(GNN_Model.parameters(), lr=classifier_lr, weight_decay=classifier_weight_decay)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
def loss_calculations(preds, gtruth):
    loss_per_epoch = criterion(preds, gtruth)
    return loss_per_epoch

In [ ]:
def visualize_losses(GNN_Model_losses, epoch_history):
    GNN_Model_losses_list = torch.stack(GNN_Model_losses).cpu().detach().numpy()

    fig = plt.figure(figsize=(27,20))

    ax = plt.subplot2grid((3, 1), (0, 0), colspan=1)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(" Loss in Epoch: " + str(epoch_history))

    ax.plot(GNN_Model_losses_list, color='r')

    #plt.savefig('/content/drive/My Drive/Explainability Methods/' + str(Explainability_name)+' on ' + str(Task_name) + '/Experimental Results/' + File_Name + 'Loss_til_epoch_{:04d}.png'.format(epoch_history))
    plt.show()

In [ ]:
def train_step(data):
    GNN_Model_loss_batch = []
    Pred_Labels = []
    Real_Labels = []

    GNN_Model.train()
    GNN_Model.zero_grad()
    for batch_of_graphs in data:
        if GNN_Model.__class__.__name__ == "GCN_plus_GAP":
            Output_of_Hidden_Layers, pooling_layer_output, ffn_output, soft = GNN_Model(batch_of_graphs)
            batch_loss = loss_calculations(soft, batch_of_graphs.y)
            Pred_Labels.extend(soft.argmax(dim=1).detach().tolist())

        elif GNN_Model.__class__.__name__ == "DGCNN_Model":
            final_GNN_layer_output, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, output_h1, dropout_output_h1, output_h2, softmaxed_h2 = GNN_Model(batch_of_graphs, None)
            batch_loss = loss_calculations(softmaxed_h2, batch_of_graphs.y)
            Pred_Labels.extend(softmaxed_h2.argmax(dim=1).detach().tolist())

        elif GNN_Model.__class__.__name__ == "DIFFPOOL_Model":
            concatination_list_of_poolings, prediction_output_not_softed, prediction_output = GNN_Model(batch_of_graphs, None)
            Pred_Labels.extend(prediction_output.argmax(dim=1).detach().tolist())
            batch_loss = loss_calculations(prediction_output, batch_of_graphs.y)

        elif GNN_Model.__class__.__name__ == "GIN_Model":
            mlps_output_embeds, mlp_outputs_globalSUMpooled, lin1_output, lin1_output_dropouted, lin2_output, lin2_output_softmaxed = GNN_Model(batch_of_graphs, None)
            batch_loss = loss_calculations(lin2_output_softmaxed, batch_of_graphs.y)
            Pred_Labels.extend(lin2_output_softmaxed.argmax(dim=1).detach().tolist())
        else:
            raise Exception("We cover GCN_plus_GAP, DGCNN, DIFFPOOL, and GIN.")

        Real_Labels.extend(batch_of_graphs.y.detach().tolist())
        GNN_Model_loss_batch.append(batch_loss)

        batch_loss.backward()
        GNN_Model_Optimizer.step()

    return torch.mean(torch.tensor(GNN_Model_loss_batch)), metrics.accuracy_score(Real_Labels, Pred_Labels)

In [ ]:
GNN_Model_training_Acc_per_epoch = []
GNN_Model_training_time_per_epoch = []
def train(EPOCHS, load_index, data):
    GNN_Model_training_loss_per_epoch = []

    for epoch in range(EPOCHS):
        t1 = perf_counter()
        GNN_Model_training_loss, training_acc = train_step(data)
        GNN_Model_training_time_per_epoch.append(perf_counter()-t1)
        print(f'Epoch: {epoch+1:03d}, Model Loss: {GNN_Model_training_loss:.4f}, Accuracy: {training_acc:.2f}')

        GNN_Model_training_loss_per_epoch.append(GNN_Model_training_loss)
        GNN_Model_training_Acc_per_epoch.append(training_acc)
        #break

        if (epoch + load_index + 1) % 50 == 0 and epoch > 0:
            visualize_losses(GNN_Model_training_loss_per_epoch, epoch + load_index + 1)
        #if (epoch + load_index + 1) % 100 == 0 and epoch > 0:
        #  torch.save({'epoch': epoch+load_index+1, 'model_state_dict': GNN_Model.state_dict(), 'optimizer_state_dict': GNN_Model_Optimizer.state_dict(), 'loss': GNN_Model_training_loss_per_epoch,}, "/content/drive/My Drive/Explainability Methods/" + str(Explainability_name) + " on " + str(Task_name) + "/Model/" + File_Name + str(epoch + load_index + 1)+".pt")



In [ ]:
EPOCHS = 1
load_index = 0

train(EPOCHS, load_index, nci1_train_dataloader)

Epoch: 001, Model Loss: 0.8144, Accuracy: 0.50


In [ ]:
class one_model_one_explainer:
    def __init__(self, dataset_name):

        self.dataset_name = dataset_name
        self.fid_plus_threshold = 0.01
        self.fid_minus_threshold = 0.01
        self.contrastivity_threshold = 0.5
        self.sparsity_threshold = 0.5
        self.stability_threshold = 0.5
        self.stability_perturbation_mean = 0.1
        self.stability_perturbation_std = 0.1


    def __call__(self, a_trained_GNN_Model, explainer_name, test_dataset, num_classes, explainer_epoch, explainer_learning_rate):


        print("Name of the Dataset: ", self.dataset_name)
        print("Number of Classes: ", num_classes)
        print("Test Dataset Size: ", len(test_dataset))
        print("Explainer: ", explainer_name)
        print("Explainer Training Epochs: ", explainer_epoch)
        print("Explainer Learning Rate: ", explainer_learning_rate)

        ##########################################################################################################################

        if explainer_name == "GNNExplainer":
            import sys
            py_path = '/content/drive/MyDrive/Explainability Methods/GNNExplainer on Graph Classification/Script/'
            sys.path.insert(0,py_path)
            import gnnexplainer_on_graph_classification as GNNExplainer


            explanations = {}
            for i in range(num_classes):
                explanations[i] = []
            t1 = perf_counter()
            for i, graph in tqdm(enumerate(test_dataset)):
                for class_index in range(num_classes):
                    EXP = GNNExplainer.GNNExplainer(a_trained_GNN_Model, explainer_epoch, explainer_learning_rate)
                    node_mask, edge_mask = EXP(graph, class_index)
                    explanations[class_index].append(node_mask)

            timing = perf_counter()-t1
            average_explanation_time = (timing)/(explainer_epoch*len(test_dataset))
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",  explainer_name, " average_explanation_time: ", average_explanation_time)

            #  Fidelity Plus
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_fidelity_plus as eval_xai_fid_plus
            eval_xai_fid_plus = reload(eval_xai_fid_plus)

            fid_plus_xmethod_example = eval_xai_fid_plus.evalaution_of_xmethods_fidelity_plus(a_trained_model=a_trained_GNN_Model,
                                                                                              test_data=test_dataset)
            fid_plus_score = fid_plus_xmethod_example.my_fidelity(saliencies_for_multiple_classes=explanations,
                                                                  importance_threshold=self.fid_plus_threshold, style="Node")
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Fid+: ", fid_plus_score)


            #  Fidelity Minus
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_fidelity_minus as eval_xai_fid_minus
            eval_xai_fid_minus = reload(eval_xai_fid_minus)

            fid_minus_xmethod_example = eval_xai_fid_minus.evalaution_of_xmethods_fidelity_minus(a_trained_model=a_trained_GNN_Model,
                                                                                                 test_data=test_dataset)
            fid_minus_score = fid_minus_xmethod_example.my_fidelity(saliencies_for_multiple_classes=explanations,
                                                                    importance_threshold=self.fid_minus_threshold, style="Node")
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Fid-: ", fid_minus_score)


            #  Contrastivity
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_contrastivity as eval_xai_contrastivity
            eval_xai_contrastivity = reload(eval_xai_contrastivity)

            contrastivity_xmethod_example = eval_xai_contrastivity.evalaution_of_xmethods_contrastivity(a_trained_model=a_trained_GNN_Model,
                                                                                                        test_data=test_dataset)
            contrastivity_score = contrastivity_xmethod_example.my_contrastivity(your_dataset=test_dataset,
                                                                                 saliencies_for_multiple_classes=explanations,
                                                                                 importance_threshold=self.contrastivity_threshold, contrast_coeff=1e+11)
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Contrastivity_Score: ", contrastivity_score)


            #  Sparsity
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_sparsity as eval_xai_sparsity
            eval_xai_sparsity = reload(eval_xai_sparsity)

            sparsity_xmethod_example = eval_xai_sparsity.evalaution_of_xmethods_sparsity(a_trained_model=a_trained_GNN_Model,
                                                                                         test_data=test_dataset)
            sparsity_score = sparsity_xmethod_example.my_sparsity(your_dataset=test_dataset, saliencies_for_multiple_classes=explanations,
                                                                  importance_threshold=self.sparsity_threshold)
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Sparsity_Score: ", sparsity_score)


            #  Stability
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_stability as eval_xai_stability
            eval_xai_stability = reload(eval_xai_stability)

            stability = eval_xai_stability.evalaution_of_xmethods_stability(a_trained_model=a_trained_GNN_Model, test_data=test_dataset)
            perturbed_test_dataset = stability.perturb_node_features_of_dataset(test_dataset, self.stability_perturbation_mean, self.stability_perturbation_std)

            explanations_for_perturbed_data = {}
            for i in range(num_classes):
                explanations_for_perturbed_data[i] = []
            for graph in perturbed_test_dataset:
                for class_index in range(num_classes):
                    EXP = GNNExplainer.GNNExplainer(a_trained_GNN_Model, explainer_epoch, explainer_learning_rate)
                    node_mask, edge_mask = EXP(graph, class_index)
                    explanations_for_perturbed_data[class_index].append(node_mask)

            stability_score = stability.my_stability(normal_saliencies_for_multiple_classes=explanations,
                                                     perturbed_saliencies_for_multiple_classes=explanations_for_perturbed_data,
                                                     top_k_features=2, importance_threshold=self.stability_threshold, style='Node')
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Stability_Score: ", stability_score)





            ######################################################################################################################

        elif explainer_name == "PGExplainer":

            import sys
            py_path = '/content/drive/MyDrive/Explainability Methods/PGExplainer on Graph Classification/Script/'
            sys.path.insert(0,py_path)
            import pgexplainer_on_graph_classification as PGExplainer_Module
            PGExplainer_Module = reload(PGExplainer_Module)

            explanations_list = {}
            explanations_tensor = {}
            for i in range(num_classes):
                explanations_list[i] = []
                explanations_tensor[i] = []
            pgex_t1 = perf_counter()
            for class_index in tqdm(range(num_classes)):

                ExTrain_or_ExTest = 'train'
                EXP = PGExplainer_Module.PGExplainer(GNN_Model=a_trained_GNN_Model, explainer_save_index=explainer_epoch,
                                                     Exp_Epoch=explainer_epoch, Exp_lr=explainer_learning_rate,
                                                     node_feat_dim=test_dataset[0].x[0].size()[0])

                EXP(ExTrain_or_ExTest=ExTrain_or_ExTest, Exp_Load_index=explainer_epoch, your_dataset=test_dataset,
                    target_class=class_index)

                ExTrain_or_ExTest = 'test'
                EXP = PGExplainer_Module.PGExplainer(GNN_Model=a_trained_GNN_Model, explainer_save_index=explainer_epoch,
                                                     Exp_Epoch=explainer_epoch, Exp_lr=explainer_learning_rate,
                                                     node_feat_dim=test_dataset[0].x[0].size()[0])
                for graph in test_dataset:
                    edge_mask = EXP(ExTrain_or_ExTest=ExTrain_or_ExTest, Exp_Load_index=explainer_epoch, your_dataset=[graph],
                                     target_class=class_index)
                    explanations_list[class_index].append(edge_mask[0].tolist())
                    explanations_tensor[class_index].append(edge_mask[0])
            pgex_timing = perf_counter()-pgex_t1
            average_explanation_time = (pgex_timing)/(explainer_epoch*len(test_dataset))
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",  explainer_name, " average_explanation_time: ", average_explanation_time)

            #  Fidelity Plus
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_fidelity_plus as eval_xai_fid_plus
            eval_xai_fid_plus = reload(eval_xai_fid_plus)

            fid_plus_xmethod_example = eval_xai_fid_plus.evalaution_of_xmethods_fidelity_plus(a_trained_model=a_trained_GNN_Model,
                                                                                              test_data=test_dataset)
            fid_plus_score = fid_plus_xmethod_example.my_fidelity(saliencies_for_multiple_classes=explanations_tensor,
                                                                  importance_threshold=self.fid_plus_threshold, style="Edge")
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Fid+: ", fid_plus_score)


            #  Fidelity Minus
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_fidelity_minus as eval_xai_fid_minus
            eval_xai_fid_minus = reload(eval_xai_fid_minus)

            fid_minus_xmethod_example = eval_xai_fid_minus.evalaution_of_xmethods_fidelity_minus(a_trained_model=a_trained_GNN_Model,
                                                                                                 test_data=test_dataset)
            fid_minus_score = fid_minus_xmethod_example.my_fidelity(saliencies_for_multiple_classes=explanations_tensor,
                                                                    importance_threshold=self.fid_minus_threshold, style="Edge")
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Fid-: ", fid_minus_score)


            #  Contrastivity
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_contrastivity as eval_xai_contrastivity
            eval_xai_contrastivity = reload(eval_xai_contrastivity)

            contrastivity_xmethod_example = eval_xai_contrastivity.evalaution_of_xmethods_contrastivity(a_trained_model=a_trained_GNN_Model,
                                                                                                        test_data=test_dataset)
            contrastivity_score = contrastivity_xmethod_example.my_contrastivity(your_dataset=test_dataset,
                                                                                 saliencies_for_multiple_classes=explanations_list,
                                                                                 importance_threshold=self.contrastivity_threshold,
                                                                                 contrast_coeff=1e+11)
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Contrastivity_Score: ", contrastivity_score)


            #  Sparsity
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_sparsity as eval_xai_sparsity
            eval_xai_sparsity = reload(eval_xai_sparsity)

            sparsity_xmethod_example = eval_xai_sparsity.evalaution_of_xmethods_sparsity(a_trained_model=a_trained_GNN_Model,
                                                                                         test_data=test_dataset)
            sparsity_score = sparsity_xmethod_example.my_sparsity(your_dataset=test_dataset,
                                                                  saliencies_for_multiple_classes=explanations_list,
                                                                  importance_threshold=self.sparsity_threshold)
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Sparsity_Score: ", sparsity_score)


            #  Stability
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_stability as eval_xai_stability
            eval_xai_stability = reload(eval_xai_stability)

            stability = eval_xai_stability.evalaution_of_xmethods_stability(a_trained_model=a_trained_GNN_Model, test_data=test_dataset)
            perturbed_test_dataset = stability.perturb_node_features_of_dataset(test_dataset, self.stability_perturbation_mean, self.stability_perturbation_std)

            explanations_for_perturbed_data_tensor = {}
            explanations_for_perturbed_data_list = {}
            for i in range(num_classes):
                explanations_for_perturbed_data_tensor[i] = []
                explanations_for_perturbed_data_list[i] = []

            for class_index in range(num_classes):

                ExTrain_or_ExTest = 'train'
                EXP = PGExplainer_Module.PGExplainer(GNN_Model=a_trained_GNN_Model, explainer_save_index=explainer_epoch,
                                                     Exp_Epoch=explainer_epoch, Exp_lr=explainer_learning_rate,
                                                     node_feat_dim=test_dataset[0].x[0].size()[0])

                EXP(ExTrain_or_ExTest=ExTrain_or_ExTest, Exp_Load_index=explainer_epoch, your_dataset=test_dataset,
                    target_class=class_index)

                ExTrain_or_ExTest = 'test'
                EXP = PGExplainer_Module.PGExplainer(GNN_Model=a_trained_GNN_Model, explainer_save_index=explainer_epoch,
                                                     Exp_Epoch=explainer_epoch, Exp_lr=explainer_learning_rate,
                                                     node_feat_dim=test_dataset[0].x[0].size()[0])
                for graph in perturbed_test_dataset:
                    edge_mask = EXP(ExTrain_or_ExTest=ExTrain_or_ExTest, Exp_Load_index=explainer_epoch, your_dataset=[graph],
                                     target_class=class_index)
                    explanations_for_perturbed_data_tensor[class_index].append(edge_mask[0])
                    explanations_for_perturbed_data_list[class_index].append(edge_mask[0].tolist())

            stability_score = stability.my_stability(normal_saliencies_for_multiple_classes=explanations_list,
                                                     perturbed_saliencies_for_multiple_classes=explanations_for_perturbed_data_list,
                                                     top_k_features=2, importance_threshold=self.stability_threshold, style='Edge')
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Stability_Score: ", stability_score)



            ######################################################################################################################

        elif explainer_name == "GraphMask":

            import sys
            py_path = '/content/drive/MyDrive/Explainability Methods/GraphMask on Graph Classification/Script/'
            sys.path.insert(0,py_path)
            import graphmask_on_graph_classification as GraphMask_Module
            GraphMask_Module = reload(GraphMask_Module)

            explanations_list = {}
            explanations_tensor = {}
            for i in range(num_classes):
                explanations_list[i] = []
                explanations_tensor[i] = []
            graphmask_t1 = perf_counter()
            for class_index in tqdm(range(num_classes)):

                ExTrain_or_ExTest = 'train'
                EXP = GraphMask_Module.GraphMask(GNN_Model=a_trained_GNN_Model, explainer_save_index=explainer_epoch,
                                                 Exp_Epoch=explainer_epoch, Exp_lr=0.001, explainer_hid_dim=test_dataset[0].x[0].size()[0],
                                                 node_feat_dim=test_dataset[0].x[0].size()[0])

                EXP(ExTrain_or_ExTest=ExTrain_or_ExTest, Exp_Load_index=explainer_epoch, your_dataset=test_dataset,
                    target_class=class_index)

                ExTrain_or_ExTest = 'test'
                EXP = GraphMask_Module.GraphMask(GNN_Model=a_trained_GNN_Model, explainer_save_index=explainer_epoch,
                                                 Exp_Epoch=explainer_epoch, Exp_lr=0.001, explainer_hid_dim=test_dataset[0].x[0].size()[0],
                                                 node_feat_dim=test_dataset[0].x[0].size()[0])
                for graph in test_dataset:
                    edge_mask = EXP(ExTrain_or_ExTest=ExTrain_or_ExTest, Exp_Load_index=explainer_epoch, your_dataset=[graph],
                                     target_class=class_index)
                    explanations_list[class_index].append(edge_mask[0].tolist())
                    explanations_tensor[class_index].append(edge_mask[0])
            graphmask_timing = perf_counter()-graphmask_t1
            average_explanation_time = (graphmask_timing)/(explainer_epoch*len(test_dataset))
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",  explainer_name, " average_explanation_time: ", average_explanation_time)


            #  Fidelity Plus
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_fidelity_plus as eval_xai_fid_plus
            eval_xai_fid_plus = reload(eval_xai_fid_plus)

            fid_plus_xmethod_example = eval_xai_fid_plus.evalaution_of_xmethods_fidelity_plus(a_trained_model=a_trained_GNN_Model,
                                                                                              test_data=test_dataset)
            fid_plus_score = fid_plus_xmethod_example.my_fidelity(saliencies_for_multiple_classes=explanations_tensor,
                                                                  importance_threshold=self.fid_plus_threshold, style="Edge")
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Fid+: ", fid_plus_score)


            #  Fidelity Minus
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_fidelity_minus as eval_xai_fid_minus
            eval_xai_fid_minus = reload(eval_xai_fid_minus)

            fid_minus_xmethod_example = eval_xai_fid_minus.evalaution_of_xmethods_fidelity_minus(a_trained_model=a_trained_GNN_Model,
                                                                                                 test_data=test_dataset)
            fid_minus_score = fid_minus_xmethod_example.my_fidelity(saliencies_for_multiple_classes=explanations_tensor,
                                                                    importance_threshold=self.fid_minus_threshold, style="Edge")
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Fid-: ", fid_minus_score)


            #  Contrastivity
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_contrastivity as eval_xai_contrastivity
            eval_xai_contrastivity = reload(eval_xai_contrastivity)

            contrastivity_xmethod_example = eval_xai_contrastivity.evalaution_of_xmethods_contrastivity(a_trained_model=a_trained_GNN_Model,
                                                                                                        test_data=test_dataset)
            contrastivity_score = contrastivity_xmethod_example.my_contrastivity(your_dataset=test_dataset,
                                                                                 saliencies_for_multiple_classes=explanations_list,
                                                                                 importance_threshold=self.contrastivity_threshold,
                                                                                 contrast_coeff=1e+11)
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Contrastivity_Score: ", contrastivity_score)


            #  Sparsity
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_sparsity as eval_xai_sparsity
            eval_xai_sparsity = reload(eval_xai_sparsity)

            sparsity_xmethod_example = eval_xai_sparsity.evalaution_of_xmethods_sparsity(a_trained_model=a_trained_GNN_Model,
                                                                                         test_data=test_dataset)
            sparsity_score = sparsity_xmethod_example.my_sparsity(your_dataset=test_dataset,
                                                                  saliencies_for_multiple_classes=explanations_list,
                                                                  importance_threshold=self.sparsity_threshold)
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Sparsity_Score: ", sparsity_score)



            #  Stability
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_stability as eval_xai_stability
            eval_xai_stability = reload(eval_xai_stability)

            stability = eval_xai_stability.evalaution_of_xmethods_stability(a_trained_model=a_trained_GNN_Model, test_data=test_dataset)
            perturbed_test_dataset = stability.perturb_node_features_of_dataset(test_dataset, self.stability_perturbation_mean, self.stability_perturbation_std)

            explanations_for_perturbed_data_tensor = {}
            explanations_for_perturbed_data_list = {}
            for i in range(num_classes):
                explanations_for_perturbed_data_tensor[i] = []
                explanations_for_perturbed_data_list[i] = []

            for class_index in range(num_classes):

                ExTrain_or_ExTest = 'train'
                EXP = GraphMask_Module.GraphMask(GNN_Model=a_trained_GNN_Model, explainer_save_index=explainer_epoch,
                                                 Exp_Epoch=explainer_epoch, Exp_lr=0.001, explainer_hid_dim=test_dataset[0].x[0].size()[0],
                                                 node_feat_dim=test_dataset[0].x[0].size()[0])

                EXP(ExTrain_or_ExTest=ExTrain_or_ExTest, Exp_Load_index=explainer_epoch, your_dataset=test_dataset,
                    target_class=class_index)

                ExTrain_or_ExTest = 'test'
                EXP = GraphMask_Module.GraphMask(GNN_Model=a_trained_GNN_Model, explainer_save_index=explainer_epoch,
                                                 Exp_Epoch=explainer_epoch, Exp_lr=0.001, explainer_hid_dim=test_dataset[0].x[0].size()[0],
                                                 node_feat_dim=test_dataset[0].x[0].size()[0])
                for graph in perturbed_test_dataset:
                    edge_mask = EXP(ExTrain_or_ExTest=ExTrain_or_ExTest, Exp_Load_index=explainer_epoch, your_dataset=[graph],
                                     target_class=class_index)
                    explanations_for_perturbed_data_tensor[class_index].append(edge_mask[0])
                    explanations_for_perturbed_data_list[class_index].append(edge_mask[0].tolist())

            stability_score = stability.my_stability(normal_saliencies_for_multiple_classes=explanations_list,
                                                     perturbed_saliencies_for_multiple_classes=explanations_for_perturbed_data_list,
                                                     top_k_features=2, importance_threshold=self.stability_threshold, style='Edge')
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Stability_Score: ", stability_score)





            ######################################################################################################################

        elif explainer_name == "SubGraphX":
            import sys
            py_path = '/content/drive/MyDrive/Explainability Methods/SubGraphX on Graph Classification/Script/'
            sys.path.insert(0,py_path)
            import subgraphx_on_graph_classification_final_format as SubGraphX_Module
            SubGraphX_Module = reload(SubGraphX_Module)

            Train_or_Test='test'
            if Train_or_Test == 'train':

                SubGraphX_t1 = perf_counter()
                SubGX = SubGraphX_Module.SubGraphX(GNN_Model=GNN_Model, num_classes=num_classes, num_hops=2, explain_graph=True,
                                                   rollout_count=20, min_children_threshold=5, ubc1_c_coef= 10.0,
                                                   expand_count_threshold=5, high2low=True, sample_num=100,
                                                   save_permission_explanations= True, dataset_name=self.dataset_name)
                for i, graph in enumerate(test_dataset):
                    explanation_results, related_preds, time_list = SubGX(graph=graph, graph_index=i)

                SubGraphX_timing = perf_counter()-SubGraphX_t1
                average_explanation_time = (SubGraphX_timing)/(len(test_dataset))
                print(a_trained_GNN_Model.__class__.__name__, " Model by ",  explainer_name, " average_explanation_time: ", average_explanation_time)

            elif Train_or_Test == 'test':
                subgx_out_the_fly = SubGraphX_Module.SubGraphX_off_the_fly(test_dataset=test_dataset, Task_name='Graph Classification',
                                                                           num_classes=num_classes, dataset_name=self.dataset_name,
                                                                           Model_Name=GNN_Model.__class__.__name__)
                whole_data = subgx_out_the_fly(test_dataset)

                explanations = {}
                for i in range(num_classes):
                    explanations[i] = []
                for cls in range(num_classes):
                    for i in range(len(test_dataset)):
                        explanations[cls].append(torch.sum(whole_data[i][cls]['important_nodes'].x, dim=1).tolist())




            #  Fidelity Plus
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_fidelity_plus as eval_xai_fid_plus
            eval_xai_fid_plus = reload(eval_xai_fid_plus)

            fid_plus_xmethod_example = eval_xai_fid_plus.evalaution_of_xmethods_fidelity_plus(a_trained_model=a_trained_GNN_Model,
                                                                                              test_data=test_dataset)
            fid_plus_score = fid_plus_xmethod_example.my_fidelity(saliencies_for_multiple_classes=explanations,
                                                                  importance_threshold=self.fid_plus_threshold, style="Node")
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Fid+: ", fid_plus_score)


            #  Fidelity Minus
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_fidelity_minus as eval_xai_fid_minus
            eval_xai_fid_minus = reload(eval_xai_fid_minus)

            fid_minus_xmethod_example = eval_xai_fid_minus.evalaution_of_xmethods_fidelity_minus(a_trained_model=a_trained_GNN_Model,
                                                                                                 test_data=test_dataset)
            fid_minus_score = fid_minus_xmethod_example.my_fidelity(saliencies_for_multiple_classes=explanations,
                                                                    importance_threshold=self.fid_minus_threshold, style="Node")
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Fid-: ", fid_minus_score)


            #  Contrastivity
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_contrastivity as eval_xai_contrastivity
            eval_xai_contrastivity = reload(eval_xai_contrastivity)

            contrastivity_xmethod_example = eval_xai_contrastivity.evalaution_of_xmethods_contrastivity(a_trained_model=a_trained_GNN_Model,
                                                                                                        test_data=test_dataset)
            contrastivity_score = contrastivity_xmethod_example.my_contrastivity(your_dataset=test_dataset,
                                                                                 saliencies_for_multiple_classes=explanations,
                                                                                 importance_threshold=self.contrastivity_threshold,
                                                                                 contrast_coeff=1e+11)
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Contrastivity_Score: ", contrastivity_score)


            #  Sparsity
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_sparsity as eval_xai_sparsity
            eval_xai_sparsity = reload(eval_xai_sparsity)

            sparsity_xmethod_example = eval_xai_sparsity.evalaution_of_xmethods_sparsity(a_trained_model=a_trained_GNN_Model,
                                                                                         test_data=test_dataset)
            sparsity_score = sparsity_xmethod_example.my_sparsity(your_dataset=test_dataset,
                                                                  saliencies_for_multiple_classes=explanations,
                                                                  importance_threshold=self.sparsity_threshold)
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Sparsity_Score: ", sparsity_score)




            #  Stability
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_stability as eval_xai_stability
            eval_xai_stability = reload(eval_xai_stability)

            stability = eval_xai_stability.evalaution_of_xmethods_stability(a_trained_model=a_trained_GNN_Model, test_data=test_dataset)
            perturbed_test_dataset = stability.perturb_node_features_of_dataset(test_dataset, self.stability_perturbation_mean,
                                                                                self.stability_perturbation_std)

            explanations_for_perturbed_data = {}
            for i in range(num_classes):
                explanations_for_perturbed_data[i] = []

            Train_or_Test_perturbed='train'
            if Train_or_Test_perturbed == 'train':
                SubGX = SubGraphX_Module.SubGraphX(GNN_Model=GNN_Model, num_classes=num_classes, num_hops=2, explain_graph=True,
                                                rollout_count=20, min_children_threshold=5, ubc1_c_coef= 10.0,
                                                expand_count_threshold=5, high2low=True, sample_num=100,
                                                save_permission_explanations= True, dataset_name="Perturbed_"+str(self.dataset_name))
                for i, graph in enumerate(perturbed_test_dataset):
                    explanation_results, related_preds, time_list = SubGX(graph=graph, graph_index=i)
            elif Train_or_Test_perturbed == 'test':
                subgx_out_the_fly = SubGraphX_Module.SubGraphX_off_the_fly(test_dataset=perturbed_test_dataset, Task_name='Graph Classification',
                                                                           num_classes=num_classes, dataset_name="Perturbed_"+str(self.dataset_name),
                                                                           Model_Name=GNN_Model.__class__.__name__)
                whole_data = subgx_out_the_fly(perturbed_test_dataset)

                for cls in range(num_classes):
                    for i in range(len(perturbed_test_dataset)):
                        explanations_for_perturbed_data[cls].append(torch.sum(whole_data[i][cls]['important_nodes'].x, dim=1).tolist())

            # print("explanations: ", explanations)
            # print("explanations_for_perturbed_data: ", explanations_for_perturbed_data)
            # for cls in explanations.keys():
            #     l1=[]
            #     for graph in explanations[cls]:
            #         l1.append(len(graph))
            #     print(l1)
            # for cls in explanations_for_perturbed_data.keys():
            #     l1=[]
            #     for graph in explanations_for_perturbed_data[cls]:
            #         l1.append(len(graph))
            #     print(l1)
            stability_score = stability.my_stability(normal_saliencies_for_multiple_classes=explanations,
                                                     perturbed_saliencies_for_multiple_classes=explanations_for_perturbed_data,
                                                     top_k_features=2, importance_threshold=self.stability_threshold, style='Node')
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Stability_Score: ", stability_score)


            ######################################################################################################################

        elif explainer_name == "CF2":
            import sys
            py_path = '/content/drive/MyDrive/Explainability Methods/CF2 on Graph Classification/Script/'
            sys.path.insert(0,py_path)
            import cf2_on_graph_classification_final_format as CF2_Module
            CF2_Module = reload(CF2_Module)
            explanations = {}
            explanations_list = {}
            for i in range(num_classes):
                explanations[i] = []
                explanations_list[i] = []
            CF2_t1 = perf_counter()
            for i in range(num_classes):
                cf2_explanation = CF2_Module.CF2_Explaination(GNN_Model=a_trained_GNN_Model, your_dataset=test_dataset,
                                                              explainer_epochs=explainer_epoch, fix_exp=None, input_dim=test_dataset[0].x[0].size()[0],
                                                              hid_dim=test_dataset[0].x[0].size()[0], output_dim=num_classes)
                exp_dict = cf2_explanation.explain_nodes_gnn_stats(category=i)
                for key, value in exp_dict.items():
                    explanations[i].append(value)
                    explanations_list[i].append(value.tolist())
            CF2_timing = perf_counter()-CF2_t1
            average_explanation_time = (CF2_timing)/(explainer_epoch*len(test_dataset))
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",  explainer_name, " average_explanation_time: ", average_explanation_time)


            #  Fidelity Plus
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_fidelity_plus as eval_xai_fid_plus
            eval_xai_fid_plus = reload(eval_xai_fid_plus)

            fid_plus_xmethod_example = eval_xai_fid_plus.evalaution_of_xmethods_fidelity_plus(a_trained_model=a_trained_GNN_Model,
                                                                                              test_data=test_dataset)
            fid_plus_score = fid_plus_xmethod_example.my_fidelity(saliencies_for_multiple_classes=explanations,
                                                                  importance_threshold=self.fid_plus_threshold, style="Edge")
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Fid+: ", fid_plus_score)


            #  Fidelity Minus
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_fidelity_minus as eval_xai_fid_minus
            eval_xai_fid_minus = reload(eval_xai_fid_minus)

            fid_minus_xmethod_example = eval_xai_fid_minus.evalaution_of_xmethods_fidelity_minus(a_trained_model=a_trained_GNN_Model,
                                                                                                 test_data=test_dataset)
            fid_minus_score = fid_minus_xmethod_example.my_fidelity(saliencies_for_multiple_classes=explanations,
                                                                    importance_threshold=self.fid_minus_threshold, style="Edge")
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Fid-: ", fid_minus_score)


            #  Contrastivity
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_contrastivity as eval_xai_contrastivity
            eval_xai_contrastivity = reload(eval_xai_contrastivity)

            contrastivity_xmethod_example = eval_xai_contrastivity.evalaution_of_xmethods_contrastivity(a_trained_model=a_trained_GNN_Model,
                                                                                                        test_data=test_dataset)
            contrastivity_score = contrastivity_xmethod_example.my_contrastivity(your_dataset=test_dataset,
                                                                                 saliencies_for_multiple_classes=explanations_list,
                                                                                 importance_threshold=self.contrastivity_threshold,
                                                                                 contrast_coeff=1e+11)
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Contrastivity_Score: ", contrastivity_score)


            #  Sparsity
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_sparsity as eval_xai_sparsity
            eval_xai_sparsity = reload(eval_xai_sparsity)

            sparsity_xmethod_example = eval_xai_sparsity.evalaution_of_xmethods_sparsity(a_trained_model=a_trained_GNN_Model,
                                                                                         test_data=test_dataset)
            sparsity_score = sparsity_xmethod_example.my_sparsity(your_dataset=test_dataset,
                                                                  saliencies_for_multiple_classes=explanations_list,
                                                                  importance_threshold=self.sparsity_threshold)
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Sparsity_Score: ", sparsity_score)



            #  Stability
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_stability as eval_xai_stability
            eval_xai_stability = reload(eval_xai_stability)

            stability = eval_xai_stability.evalaution_of_xmethods_stability(a_trained_model=a_trained_GNN_Model,
                                                                            test_data=test_dataset)
            perturbed_test_dataset = stability.perturb_node_features_of_dataset(test_dataset, self.stability_perturbation_mean,
                                                                                self.stability_perturbation_std)

            explanations_for_perturbed_data_tensor = {}
            explanations_for_perturbed_data_list = {}
            for i in range(num_classes):
                explanations_for_perturbed_data_tensor[i] = []
                explanations_for_perturbed_data_list[i] = []

            for i in range(num_classes):
                cf2_explanation = CF2_Module.CF2_Explaination(GNN_Model=a_trained_GNN_Model, your_dataset=perturbed_test_dataset,
                                                              explainer_epochs=explainer_epoch, fix_exp=None, input_dim=test_dataset[0].x[0].size()[0],
                                                              hid_dim=test_dataset[0].x[0].size()[0], output_dim=num_classes)
                exp_dict = cf2_explanation.explain_nodes_gnn_stats(category=i)
                for key, value in exp_dict.items():
                    explanations_for_perturbed_data_tensor[i].append(value)
                    explanations_for_perturbed_data_list[i].append(value.tolist())

            stability_score = stability.my_stability(normal_saliencies_for_multiple_classes=explanations_list,
                                                     perturbed_saliencies_for_multiple_classes=explanations_for_perturbed_data_list,
                                                     top_k_features=2, importance_threshold=self.stability_threshold, style='Edge')
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Stability_Score: ", stability_score)








            ######################################################################################################################

        elif explainer_name == "PGMExplainer":
            import sys
            py_path = '/content/drive/MyDrive/Explainability Methods/PGMExplainer on Graph Classification/Script/'
            sys.path.insert(0,py_path)
            import pgmexplainer_on_graph_classification_final_format as PGMExplainer_Module
            PGMExplainer_Module = reload(PGMExplainer_Module)

            explanations = {}
            for i in range(num_classes):
                explanations[i] = []
            PGMExplainer_t1 = perf_counter()
            for i in tqdm(range(num_classes)):
                for j in range(len(test_dataset)):

                    pgmx = PGMExplainer_Module.PGM_Graph_Explainer(GNN_Model=a_trained_GNN_Model, graph=test_dataset[j],
                                                                   perturb_feature_list=[None], perturb_mode = "mean",
                                                                   perturb_indicator = "abs")
                    pgm_node, p_values, candidate_nodes, dependent_nodes = pgmx.explain(num_samples=len(test_dataset[j].x),
                                                                                        noise_offset_percentage=50, top_node=5,
                                                                                        p_value_threshold=0.05, class_index=i)
                    test_graph = deepcopy(test_dataset[j])
                    for k in range(len(test_graph.x)):
                        if k not in pgm_node:
                            test_graph.x[k] = torch.zeros_like(test_graph.x[k])
                    graph_list = []
                    for node in test_graph.x:
                        graph_list.append(max(node).tolist())
                    explanations[i].append(graph_list)
            PGMExplainer_timing = perf_counter()-PGMExplainer_t1
            average_explanation_time = (PGMExplainer_timing)/(explainer_epoch*len(test_dataset))
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",  explainer_name, " average_explanation_time: ", average_explanation_time)

            #  Fidelity Plus
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_fidelity_plus as eval_xai_fid_plus
            eval_xai_fid_plus = reload(eval_xai_fid_plus)

            fid_plus_xmethod_example = eval_xai_fid_plus.evalaution_of_xmethods_fidelity_plus(a_trained_model=a_trained_GNN_Model,
                                                                                              test_data=test_dataset)
            fid_plus_score = fid_plus_xmethod_example.my_fidelity(saliencies_for_multiple_classes=explanations,
                                                                  importance_threshold=self.fid_plus_threshold, style="Node")
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Fid+: ", fid_plus_score)


            #  Fidelity Minus
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_fidelity_minus as eval_xai_fid_minus
            eval_xai_fid_minus = reload(eval_xai_fid_minus)

            fid_minus_xmethod_example = eval_xai_fid_minus.evalaution_of_xmethods_fidelity_minus(a_trained_model=a_trained_GNN_Model,
                                                                                                 test_data=test_dataset)
            fid_minus_score = fid_minus_xmethod_example.my_fidelity(saliencies_for_multiple_classes=explanations,
                                                                    importance_threshold=self.fid_minus_threshold, style="Node")
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Fid-: ", fid_minus_score)




            #  Contrastivity
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_contrastivity as eval_xai_contrastivity
            eval_xai_contrastivity = reload(eval_xai_contrastivity)

            contrastivity_xmethod_example = eval_xai_contrastivity.evalaution_of_xmethods_contrastivity(a_trained_model=a_trained_GNN_Model,
                                                                                                        test_data=test_dataset)
            contrastivity_score = contrastivity_xmethod_example.my_contrastivity(your_dataset=test_dataset,
                                                                                 saliencies_for_multiple_classes=explanations,
                                                                                 importance_threshold=self.contrastivity_threshold,
                                                                                 contrast_coeff=1e+11)
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Contrastivity_Score: ", contrastivity_score)


            #  Sparsity
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_sparsity as eval_xai_sparsity
            eval_xai_sparsity = reload(eval_xai_sparsity)

            sparsity_xmethod_example = eval_xai_sparsity.evalaution_of_xmethods_sparsity(a_trained_model=a_trained_GNN_Model,
                                                                                         test_data=test_dataset)
            sparsity_score = sparsity_xmethod_example.my_sparsity(your_dataset=test_dataset,
                                                                  saliencies_for_multiple_classes=explanations,
                                                                  importance_threshold=self.sparsity_threshold)
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Sparsity_Score: ", sparsity_score)


            #  Stability
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_stability as eval_xai_stability
            eval_xai_stability = reload(eval_xai_stability)

            stability = eval_xai_stability.evalaution_of_xmethods_stability(a_trained_model=a_trained_GNN_Model,
                                                                            test_data=test_dataset)
            perturbed_test_dataset = stability.perturb_node_features_of_dataset(test_dataset, self.stability_perturbation_mean,
                                                                                self.stability_perturbation_std)

            explanations_for_perturbed_data = {}
            for i in range(num_classes):
                explanations_for_perturbed_data[i] = []

            for i in range(num_classes):
                for j in range(len(perturbed_test_dataset)):

                    pgmx = PGMExplainer_Module.PGM_Graph_Explainer(GNN_Model=a_trained_GNN_Model, graph=perturbed_test_dataset[j],
                                                                   perturb_feature_list=[None], perturb_mode = "mean",
                                                                   perturb_indicator = "abs")
                    pgm_node, p_values, candidate_nodes, dependent_nodes = pgmx.explain(num_samples=len(perturbed_test_dataset[j].x),
                                                                                        noise_offset_percentage=50, top_node=5,
                                                                                        p_value_threshold=0.05, class_index=i)
                    test_graph = deepcopy(perturbed_test_dataset[j])
                    for k in range(len(test_graph.x)):
                        if k not in pgm_node:
                            test_graph.x[k] = torch.zeros_like(test_graph.x[k])
                    graph_list = []
                    for node in test_graph.x:
                        graph_list.append(max(node).tolist())
                    explanations_for_perturbed_data[i].append(graph_list)
            stability_score = stability.my_stability(normal_saliencies_for_multiple_classes=explanations,
                                            perturbed_saliencies_for_multiple_classes=explanations_for_perturbed_data,
                                            top_k_features=2, importance_threshold=self.sparsity_threshold, style='Node')
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Stability_Score: ", stability_score)







            ######################################################################################################################

        elif explainer_name == "XGNN":
            import sys
            py_path = '/content/drive/MyDrive/Explainability Methods/XGNN on Graph Classification/Script/'
            sys.path.insert(0,py_path)
            import xgnn_on_graph_classification_final_format as XGNN_Module
            XGNN_Module = reload(XGNN_Module)
            mutag_max_number_of_nodes=38
            explanations = {}
            for i in range(num_classes):
                explanations[i] = []
            xgnn_t1 = perf_counter()
            for cls_index in range(num_classes):
                xgnn_training = XGNN_Module.XGNN_training(GNN_Model=a_trained_GNN_Model, max_geneneration_iterations=10,
                                                          num_node_features=test_dataset[0].x[0].size()[0],
                                                          candidate_set_length=test_dataset[0].x[0].size()[0],
                                                          max_number_of_nodes=mutag_max_number_of_nodes,
                                                          random_start=True, rollout_count=10, class_of_explanation=cls_index,
                                                          hyp_for_rollout=1, hyp_for_rules=2, dropout_rate=0.5, explainer_lr=0.01,
                                                          b1=0.9, b2=0.999, weight_decay=5e-4)
                trained_graph = xgnn_training(explainer_epochs=explainer_epoch)
                explanation_graph = Data(x=trained_graph['feat'][:trained_graph['num_nodes']], edge_index=trained_graph['adj'], y=cls_index)

                py_path = '/content/drive/MyDrive/Explainability Methods/TopDown Approach on Graph Classification/Script/'
                sys.path.insert(0,py_path)
                import topdown_approach_for_global_methods as topdown_isomorphism_scoring
                topdown_isomorphism_scoring = reload(topdown_isomorphism_scoring)

                for i in range(len(test_dataset)):
                    common_edges_finder = topdown_isomorphism_scoring.global_explanation_and_samples_intersection(explanation=explanation_graph, input_graph=test_dataset[i])
                    intersection_edges = common_edges_finder()
                    explanations[cls_index].append(torch.from_numpy(np.array(list(intersection_edges.values()))))
            xgnn_timing = perf_counter()-xgnn_t1
            average_explanation_time = (xgnn_timing)/(explainer_epoch*len(test_dataset))
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",  explainer_name, " average_explanation_time: ", average_explanation_time)


            #  Fidelity Plus
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_fidelity_plus as eval_xai_fid_plus
            eval_xai_fid_plus = reload(eval_xai_fid_plus)

            fid_plus_xmethod_example = eval_xai_fid_plus.evalaution_of_xmethods_fidelity_plus(a_trained_model=a_trained_GNN_Model,
                                                                                              test_data=test_dataset)
            fid_plus_score = fid_plus_xmethod_example.my_fidelity(saliencies_for_multiple_classes=explanations,
                                                                  importance_threshold=self.fid_plus_threshold, style="Edge")
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Fid+: ", fid_plus_score)



            #  Fidelity Minus
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_fidelity_minus as eval_xai_fid_minus
            eval_xai_fid_minus = reload(eval_xai_fid_minus)

            fid_minus_xmethod_example = eval_xai_fid_minus.evalaution_of_xmethods_fidelity_minus(a_trained_model=a_trained_GNN_Model,
                                                                                                 test_data=test_dataset)
            fid_minus_score = fid_minus_xmethod_example.my_fidelity(saliencies_for_multiple_classes=explanations,
                                                                    importance_threshold=self.fid_minus_threshold, style="Edge")
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Fid-: ", fid_minus_score)



            #  Contrastivity
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_contrastivity as eval_xai_contrastivity
            eval_xai_contrastivity = reload(eval_xai_contrastivity)

            contrastivity_xmethod_example = eval_xai_contrastivity.evalaution_of_xmethods_contrastivity(a_trained_model=a_trained_GNN_Model,
                                                                                                        test_data=test_dataset)
            contrastivity_score = contrastivity_xmethod_example.my_contrastivity(your_dataset=test_dataset,
                                                                                 saliencies_for_multiple_classes=explanations,
                                                                                 importance_threshold=self.contrastivity_threshold,
                                                                                 contrast_coeff=1e+11)
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Contrastivity_Score: ", contrastivity_score)




            #  Sparsity
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_sparsity as eval_xai_sparsity
            eval_xai_sparsity = reload(eval_xai_sparsity)

            sparsity_xmethod_example = eval_xai_sparsity.evalaution_of_xmethods_sparsity(a_trained_model=a_trained_GNN_Model,
                                                                                         test_data=test_dataset)
            sparsity_score = sparsity_xmethod_example.my_sparsity(your_dataset=test_dataset,
                                                                  saliencies_for_multiple_classes=explanations,
                                                                  importance_threshold=self.sparsity_threshold)
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Sparsity_Score: ", sparsity_score)





            #  Stability
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_stability as eval_xai_stability
            eval_xai_stability = reload(eval_xai_stability)

            stability = eval_xai_stability.evalaution_of_xmethods_stability(a_trained_model=a_trained_GNN_Model,
                                                                            test_data=test_dataset)
            perturbed_test_dataset = stability.perturb_node_features_of_dataset(test_dataset, self.stability_perturbation_mean,
                                                                                self.stability_perturbation_std)

            explanations_for_perturbed_data = {}
            for i in range(num_classes):
                explanations_for_perturbed_data[i] = []

            for cls_index in range(num_classes):
                xgnn_training = XGNN_Module.XGNN_training(GNN_Model=a_trained_GNN_Model, max_geneneration_iterations=10,
                                                          num_node_features=test_dataset[0].x[0].size()[0],
                                                          candidate_set_length=test_dataset[0].x[0].size()[0],
                                                          max_number_of_nodes=mutag_max_number_of_nodes,
                                                          random_start=True, rollout_count=10, class_of_explanation=cls_index,
                                                          hyp_for_rollout=1, hyp_for_rules=2, dropout_rate=0.5, explainer_lr=0.01,
                                                          b1=0.9, b2=0.999, weight_decay=5e-4)
                perturbed_trained_graph = xgnn_training(explainer_epochs=explainer_epoch)
                per_explanation_graph = Data(x=perturbed_trained_graph['feat'][:perturbed_trained_graph['num_nodes']], edge_index=perturbed_trained_graph['adj'], y=cls_index)
                py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Model-level and Global Methods on Graph Classification/Script/'
                sys.path.insert(0,py_path)
                import topdown_approach_for_global_methods as topdown_isomorphism_scoring
                topdown_isomorphism_scoring = reload(topdown_isomorphism_scoring)

                for i in range(len(perturbed_test_dataset)):
                    common_edges_finder = topdown_isomorphism_scoring.global_explanation_and_samples_intersection(explanation=per_explanation_graph, input_graph=perturbed_test_dataset[i])
                    perturbed_intersection_edges = common_edges_finder()
                    explanations_for_perturbed_data[cls_index].append(torch.from_numpy(np.array(list(perturbed_intersection_edges.values()))))

            stability_score = stability.my_stability(normal_saliencies_for_multiple_classes=explanations,
                                                     perturbed_saliencies_for_multiple_classes=explanations_for_perturbed_data,
                                                     top_k_features=2, importance_threshold=self.stability_threshold, style='Edge')
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Stability_Score: ", stability_score)



            # import pickle
            # file_path = '/content/drive/My Drive/Explainability Methods/'+str(Explainability_name)+' on ' + str(Task_name) + '/Experimental Results/Edge_Limit/' + File_Name + 'Limited_Edge_Explanation_Class_' + str(class_of_explanation)

            # with open(file_path, 'wb') as file:
            #     pickle.dump(generated_graph, file)
            ######################################################################################################################

        elif explainer_name == "GNNInterpreter":
            import sys
            py_path = '/content/drive/MyDrive/Explainability Methods/GNNInterpreter on Graph Classification/Script/'
            sys.path.insert(0,py_path)
            import gnninterpreter_on_graph_classification_final_format as GNNInterpreter_Module




            explanations_size = {0:9, 1:7, 2:8}
            explanations = {}
            for i in range(num_classes):
                explanations[i] = []

            generations = {}
            for i in range(num_classes):
                generations[i] = []

            mean_of_node_embeddings = {}
            for i in range(num_classes):
                mean_of_node_embeddings[i] = torch.mean(torch.cat([graph.x for graph in test_dataset if graph.y == i], dim=0), axis=0)

            gnninterpreter_t1 = perf_counter()
            for cls_index in range(num_classes):
                losses_aggregated = GNNInterpreter_Module.losses_aggregation(
                    [
                        dict(key="continuous_generated_embeddings", criterion=GNNInterpreter_Module.Embedding_Loss_by_Cosine_Similarity(target_embedding=mean_of_node_embeddings[cls_index]), weight=10),
                        dict(key="discrete_generated_embeddings", criterion=GNNInterpreter_Module.Embedding_Loss_by_Cosine_Similarity(target_embedding=mean_of_node_embeddings[cls_index]), weight=10),
                        dict(key="logits_continuous", criterion=GNNInterpreter_Module.Explanation_Class_Score(class_idx=cls_index, mode='maximize'), weight=1),
                        dict(key="logits_continuous", criterion=GNNInterpreter_Module.MeanPenalty(), weight=0),
                        dict(key="logits_discrete", criterion=GNNInterpreter_Module.Explanation_Class_Score(class_idx=cls_index, mode='maximize'), weight=1),
                        dict(key="logits_discrete", criterion=GNNInterpreter_Module.MeanPenalty(), weight=0),
                        dict(key="edge_parameters", criterion=GNNInterpreter_Module.NormPenalty(order=1), weight=1),
                        dict(key="edge_parameters", criterion=GNNInterpreter_Module.NormPenalty(order=2), weight=1),
                        dict(key="node_feature_parameters", criterion=GNNInterpreter_Module.NormPenalty(order=1), weight=0),
                        dict(key="node_feature_parameters", criterion=GNNInterpreter_Module.NormPenalty(order=2), weight=0),
                        # dict(key="edge_feature_parameters", criterion=NormPenalty(order=1), weight=0),
                        # dict(key="edge_feature_parameters", criterion=NormPenalty(order=2), weight=0),
                        dict(key="edge_parameters_pairs_of_nodes", criterion=GNNInterpreter_Module.KLDivergencePenalty(binary=True), weight=0)
                        ]
                    )
                generator = GNNInterpreter_Module.Graph_Generator(max_nodes=explanations_size[cls_index], num_node_classes=test_dataset[0].x[0].size()[0],
                                                                  num_edge_classes=None, nodes=None, edges=None, Graph=None,
                                                                  learning_node_feat=True, learning_edge_feat=False, temperature=0.15)

                generations[cls_index] = GNNInterpreter_Module.Generation_Manager_wrt_Classes(generator=generator,
                                                                                              discriminator=a_trained_GNN_Model,
                                                                                              aggregate_losses=losses_aggregated,
                                                                                              optimizer=(o := torch.optim.SGD(generator.parameters(), lr=1)),
                                                                                              dataset=test_dataset,
                                                                                              budget_penalty=GNNInterpreter_Module.BudgetPenalty_for_second_regularization(budget=10, order=2, beta=1),
                                                                                              targeted_probabilities={cls_index: (0.9, 1)},
                                                                                              batch_size_for_same_sized_graphs=1)

                continuous_generated_graph, discrete_generated_graph = generations[cls_index].train(explainer_epoch)
                # explanations[cls_index] = discrete_generated_graph
                #print("discrete_generated_graph: ", discrete_generated_graph)

                py_path = '/content/drive/MyDrive/Explainability Methods/TopDown Approach on Graph Classification/Script/'
                sys.path.insert(0,py_path)
                import topdown_approach_for_global_methods as topdown_isomorphism_scoring
                topdown_isomorphism_scoring = reload(topdown_isomorphism_scoring)

                for i in range(len(test_dataset)):
                    common_edges_finder = topdown_isomorphism_scoring.global_explanation_and_samples_intersection(explanation=discrete_generated_graph, input_graph=test_dataset[i])
                    intersection_edges = common_edges_finder()
                    explanations[cls_index].append(torch.from_numpy(np.array(list(intersection_edges.values()))))

            gnninterpreter_timing = perf_counter()-gnninterpreter_t1
            average_explanation_time = (gnninterpreter_timing)/(explainer_epoch*len(test_dataset))
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",  explainer_name, " average_explanation_time: ", average_explanation_time)


            #  Fidelity Plus
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_fidelity_plus as eval_xai_fid_plus
            eval_xai_fid_plus = reload(eval_xai_fid_plus)

            fid_plus_xmethod_example = eval_xai_fid_plus.evalaution_of_xmethods_fidelity_plus(a_trained_model=a_trained_GNN_Model,
                                                                                              test_data=test_dataset)
            fid_plus_score = fid_plus_xmethod_example.my_fidelity(saliencies_for_multiple_classes=explanations,
                                                                  importance_threshold=self.fid_plus_threshold, style="Edge")
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Fid+: ", fid_plus_score)



            #  Fidelity Minus
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_fidelity_minus as eval_xai_fid_minus
            eval_xai_fid_minus = reload(eval_xai_fid_minus)

            fid_minus_xmethod_example = eval_xai_fid_minus.evalaution_of_xmethods_fidelity_minus(a_trained_model=a_trained_GNN_Model,
                                                                                                 test_data=test_dataset)
            fid_minus_score = fid_minus_xmethod_example.my_fidelity(saliencies_for_multiple_classes=explanations,
                                                                    importance_threshold=self.fid_minus_threshold, style="Edge")
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Fid-: ", fid_minus_score)



            #  Contrastivity
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_contrastivity as eval_xai_contrastivity
            eval_xai_contrastivity = reload(eval_xai_contrastivity)

            contrastivity_xmethod_example = eval_xai_contrastivity.evalaution_of_xmethods_contrastivity(a_trained_model=a_trained_GNN_Model,
                                                                                                        test_data=test_dataset)
            contrastivity_score = contrastivity_xmethod_example.my_contrastivity(your_dataset=test_dataset,
                                                                                 saliencies_for_multiple_classes=explanations,
                                                                                 importance_threshold=self.contrastivity_threshold,
                                                                                 contrast_coeff=1e+11)
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Contrastivity_Score: ", contrastivity_score)




            #  Sparsity
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_sparsity as eval_xai_sparsity
            eval_xai_sparsity = reload(eval_xai_sparsity)

            sparsity_xmethod_example = eval_xai_sparsity.evalaution_of_xmethods_sparsity(a_trained_model=a_trained_GNN_Model,
                                                                                         test_data=test_dataset)
            sparsity_score = sparsity_xmethod_example.my_sparsity(your_dataset=test_dataset,
                                                                  saliencies_for_multiple_classes=explanations,
                                                                  importance_threshold=self.sparsity_threshold)
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Sparsity_Score: ", sparsity_score)




            #  Stability
            py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Explainability Methods/Script/'
            sys.path.insert(0,py_path)
            import evaluation_of_xmethods_stability as eval_xai_stability
            eval_xai_stability = reload(eval_xai_stability)

            generations_by_perturbed_data = {}
            for i in range(num_classes):
                generations_by_perturbed_data[i] = []

            stability = eval_xai_stability.evalaution_of_xmethods_stability(a_trained_model=a_trained_GNN_Model,
                                                                            test_data=test_dataset)
            perturbed_test_dataset = stability.perturb_node_features_of_dataset(test_dataset, self.stability_perturbation_mean,
                                                                                self.stability_perturbation_std)

            explanations_for_perturbed_data = {}
            for i in range(num_classes):
                explanations_for_perturbed_data[i] = []

            for cls_index in range(num_classes):
                losses_aggregated = GNNInterpreter_Module.losses_aggregation(
                    [
                        dict(key="continuous_generated_embeddings", criterion=GNNInterpreter_Module.Embedding_Loss_by_Cosine_Similarity(target_embedding=mean_of_node_embeddings[cls_index]), weight=10),
                        dict(key="discrete_generated_embeddings", criterion=GNNInterpreter_Module.Embedding_Loss_by_Cosine_Similarity(target_embedding=mean_of_node_embeddings[cls_index]), weight=10),
                        dict(key="logits_continuous", criterion=GNNInterpreter_Module.Explanation_Class_Score(class_idx=cls_index, mode='maximize'), weight=1),
                        dict(key="logits_continuous", criterion=GNNInterpreter_Module.MeanPenalty(), weight=0),
                        dict(key="logits_discrete", criterion=GNNInterpreter_Module.Explanation_Class_Score(class_idx=cls_index, mode='maximize'), weight=1),
                        dict(key="logits_discrete", criterion=GNNInterpreter_Module.MeanPenalty(), weight=0),
                        dict(key="edge_parameters", criterion=GNNInterpreter_Module.NormPenalty(order=1), weight=1),
                        dict(key="edge_parameters", criterion=GNNInterpreter_Module.NormPenalty(order=2), weight=1),
                        dict(key="node_feature_parameters", criterion=GNNInterpreter_Module.NormPenalty(order=1), weight=0),
                        dict(key="node_feature_parameters", criterion=GNNInterpreter_Module.NormPenalty(order=2), weight=0),
                        # dict(key="edge_feature_parameters", criterion=NormPenalty(order=1), weight=0),
                        # dict(key="edge_feature_parameters", criterion=NormPenalty(order=2), weight=0),
                        dict(key="edge_parameters_pairs_of_nodes", criterion=GNNInterpreter_Module.KLDivergencePenalty(binary=True), weight=0)
                        ]
                    )
                generator = GNNInterpreter_Module.Graph_Generator(max_nodes=explanations_size[cls_index],
                                                                  num_node_classes=test_dataset[0].x[0].size()[0],
                                                                  num_edge_classes=4, nodes=None, edges=None, Graph=None,
                                                                  learning_node_feat=True, learning_edge_feat=False, temperature=0.15)

                generations_by_perturbed_data[cls_index] = GNNInterpreter_Module.Generation_Manager_wrt_Classes(generator=generator,
                                                                                              discriminator=a_trained_GNN_Model,
                                                                                              aggregate_losses=losses_aggregated,
                                                                                              optimizer=(o := torch.optim.SGD(generator.parameters(), lr=1)),
                                                                                              dataset=perturbed_test_dataset,
                                                                                              budget_penalty=GNNInterpreter_Module.BudgetPenalty_for_second_regularization(budget=10, order=2, beta=1),
                                                                                              targeted_probabilities={cls_index: (0.9, 1)},
                                                                                              batch_size_for_same_sized_graphs=1)

                continuous_generated_graph, discrete_generated_graph = generations_by_perturbed_data[cls_index].train(explainer_epoch)

                py_path = '/content/drive/MyDrive/Explainability Methods/Evaluation of Model-level and Global Methods on Graph Classification/Script/'
                sys.path.insert(0,py_path)
                import topdown_approach_for_global_methods as topdown_isomorphism_scoring
                topdown_isomorphism_scoring = reload(topdown_isomorphism_scoring)

                for i in range(len(perturbed_test_dataset)):
                    common_edges_finder = topdown_isomorphism_scoring.global_explanation_and_samples_intersection(explanation=discrete_generated_graph, input_graph=perturbed_test_dataset[i])
                    intersection_edges = common_edges_finder()
                    explanations_for_perturbed_data[cls_index].append(torch.from_numpy(np.array(list(intersection_edges.values()))))

            stability_score = stability.my_stability(normal_saliencies_for_multiple_classes=explanations,
                                                     perturbed_saliencies_for_multiple_classes=explanations_for_perturbed_data,
                                                     top_k_features=2, importance_threshold=self.stability_threshold, style='Edge')
            print(a_trained_GNN_Model.__class__.__name__, " Model by ",explainer_name, " Stability_Score: ", stability_score)


            ######################################################################################################################

# GCN_plus_GAP - DGCNN - DIFFPOOL - GIN
# GNNExplainer - PGExplainer - GraphMask - SubGraphX - CF2 - PGMExplainer - XGNN - GNNInterpreter
# Datasets name: MUTAG, Fake
just_call_me = one_model_one_explainer(dataset_name="NCI1")
just_call_me(a_trained_GNN_Model=GNN_Model, explainer_name="SubGraphX", test_dataset=nci1_test_dataset, num_classes=2,
             explainer_epoch=200, explainer_learning_rate=0.001)

Name of the Dataset:  NCI1
Number of Classes:  2
Test Dataset Size:  822
Explainer:  SubGraphX
Explainer Training Epochs:  200
Explainer Learning Rate:  0.001


KeyboardInterrupt: 

### **Fid+**

In [ ]:
class evalaution_of_xmethods_fidelity_plus(object):
    def __init__(self, a_trained_model, test_data):
        super(evalaution_of_xmethods_fidelity_plus, self).__init__()
        self.a_trained_model = a_trained_model
        self.test_data = test_data

    def is_salient(self, score, importance_threshold):
        if importance_threshold == score == 0:
            return True
        if importance_threshold == score == 1:
            return False
        if importance_threshold < score:
            return True
        else:
            return False

    def Compute_ROC_AUC(self, your_model, your_dataset):
        preds = []
        reals = []

        your_model.eval()
        for batched_data in your_dataset:
            if your_model.__class__.__name__ == "GCN_plus_GAP":
                Output_of_Hidden_Layers, pooling_layer_output, ffn_output, prediction = your_model(batched_data)
            elif your_model.__class__.__name__ == "DGCNN_Model":
                final_GNN_layer_output, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, output_h1, dropout_output_h1, output_h2, prediction = your_model(batched_data, None)
            elif your_model.__class__.__name__ == "DIFFPOOL_Model":
                concatination_list_of_poolings, prediction_output_not_softed, prediction = your_model(batched_data, None)
            elif your_model.__class__.__name__ == "GIN_Model":
                mlps_output_embeds, mlp_outputs_globalSUMpooled, lin1_output, lin1_output_dropouted, lin2_output, prediction = your_model(batched_data, None)
            else:
                print("Model Name is not valid.")

            # preds.extend(torch.argmax(prediction, dim=1).tolist())
            preds.extend(prediction.tolist())
            reals.extend(batched_data.y.tolist())

        is_binary = len(np.unique(reals)) == 2
        if is_binary:
            preds = (np.array(preds)[:, 1] > 0.5).astype(int)

        auc_roc = metrics.roc_auc_score(np.array(reals), np.array(preds), multi_class="ovr", average="micro")

        return auc_roc

    def drop_important_node_features(self, your_dataset, importance_threshold, attribution_scores):
        occluded_GNNgraph_list = []
        for i in range(len(attribution_scores)):
            sample_graph = deepcopy(your_dataset[i])
            for j in range(len(attribution_scores[i])):
                for k in range(len(attribution_scores[i][j])):
                    if self.is_salient(attribution_scores[i][j][k], importance_threshold):
                        sample_graph.x[j][k] = 0
            occluded_GNNgraph_list.append(sample_graph)
        return occluded_GNNgraph_list

    def drop_important_nodes(self, your_dataset, importance_threshold, attribution_scores):
        occluded_GNNgraph_list = []
        for i in range(len(attribution_scores)):
            sample_graph = deepcopy(your_dataset[i])
            for j in range(len(attribution_scores[i])):
                if self.is_salient(attribution_scores[i][j], importance_threshold):
                    sample_graph.x[j][:] = 0
            occluded_GNNgraph_list.append(sample_graph)
        return occluded_GNNgraph_list

    def Fidelity_node_features(self, your_dataset, generated_saliency_maps, importance_threshold):

        auc_roc_before_droping_important_nodes = self.Compute_ROC_AUC(self.a_trained_model, your_dataset)
        new_graph_dataset = self.drop_important_node_features(your_dataset, importance_threshold, generated_saliency_maps)
        auc_roc_after_droping_important_nodes = self.Compute_ROC_AUC(self.a_trained_model, new_graph_dataset)

        return auc_roc_before_droping_important_nodes - auc_roc_after_droping_important_nodes

    def Fidelity_node(self, your_dataset, generated_saliency_maps, importance_threshold):

        auc_roc_before_droping_important_nodes = self.Compute_ROC_AUC(self.a_trained_model, your_dataset)
        new_graph_dataset = self.drop_important_nodes(your_dataset, importance_threshold, generated_saliency_maps)
        auc_roc_after_droping_important_nodes = self.Compute_ROC_AUC(self.a_trained_model, new_graph_dataset)

        return auc_roc_before_droping_important_nodes - auc_roc_after_droping_important_nodes

    def drop_important_edges(self, your_dataset, importance_threshold, Edge_Masks_Dropped):
        attribution_scores = []
        Edge_Masks_original = []
        Edge_Masks_Dropped_copy = []
        for tensor in Edge_Masks_Dropped:
            Edge_Masks_original.append(deepcopy(tensor.detach()))
            Edge_Masks_Dropped_copy.append(deepcopy(tensor.detach()))

        for edge_mask in Edge_Masks_Dropped_copy:

            importance_indices = edge_mask > importance_threshold * (max(edge_mask)-min(edge_mask))
            edge_mask[importance_indices] = 0

        return Edge_Masks_original, Edge_Masks_Dropped_copy

    def Fidelity_edge(self, your_dataset, generated_saliency_maps, importance_threshold):

        auc_roc_before_droping_important_nodes = self.Compute_ROC_AUC_edge(self.a_trained_model, your_dataset, False)
        Edge_Masks_original, Edge_Masks_Dropped = self.drop_important_edges(your_dataset, importance_threshold, generated_saliency_maps)
        auc_roc_after_droping_important_nodes = self.Compute_ROC_AUC_edge(self.a_trained_model, your_dataset, Edge_Masks_Dropped)

        return auc_roc_before_droping_important_nodes - auc_roc_after_droping_important_nodes

    def normalize_saliency_node_features_based(self, sal_maps):
        Graphs_new_gradients = []
        for graph_grads in sal_maps:
            new_gradients = []
            for node_grads in graph_grads:
                node_gradients = []
                for dim in node_grads:
                    node_gradients.append((dim-min(node_grads))/(max(node_grads)-min(node_grads)))
                new_gradients.append(node_gradients)
            Graphs_new_gradients.append(new_gradients)
        return Graphs_new_gradients

    def normalize_saliency_node_based(self, sal_maps):
        Graphs_new_gradients = []
        for graph_grads in sal_maps:
            new_gradients = []
            for node_grads in graph_grads:
                val = (node_grads-min(graph_grads))/(max(graph_grads)-min(graph_grads)) if (max(graph_grads)-min(graph_grads)) != 0 else 0
                new_gradients.append(val)
            Graphs_new_gradients.append(new_gradients)
        return Graphs_new_gradients

    def clear_masks(self, model):

        for module in model.modules():
            if isinstance(module, MessagePassing):
                module.explain = False
                module._edge_mask = None
                module._loop_mask = None
                module._apply_sigmoid = True
        return module

    def apply_masks(self, model, mask, edge_index, apply_sigmoid):
        loop_mask = edge_index[0] != edge_index[1]

        for module in model.modules():
            if isinstance(module, MessagePassing):

                if (not isinstance(mask, Parameter)
                        and '_edge_mask' in module._parameters):
                    mask = Parameter(mask)

                module.explain = True
                module._edge_mask = mask
                module._loop_mask = loop_mask
                module._apply_sigmoid = apply_sigmoid

    def Compute_ROC_AUC_edge(self, your_model, your_dataset, masked):
        preds = []
        reals = []

        if masked == False:
            your_model.eval()
            for batched_data in your_dataset:
                reals.extend(batched_data.y.tolist())
                if your_model.__class__.__name__ == "GCN_plus_GAP":
                    Output_of_Hidden_Layers, pooling_layer_output, ffn_output, soft = your_model(batched_data)
                    preds.extend(soft.tolist())
                elif your_model.__class__.__name__ == "DGCNN_Model":
                    final_GNN_layer_output, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, output_h1, dropout_output_h1, output_h2, soft = your_model(batched_data, None)
                    preds.extend(soft.tolist())
                elif your_model.__class__.__name__ == "DIFFPOOL_Model":
                    concatination_list_of_poolings, prediction_output_not_softed, soft = your_model(batched_data, None)
                    preds.extend(soft.tolist())
                elif your_model.__class__.__name__ == "GIN_Model":
                    mlps_output_embeds, mlp_outputs_globalSUMpooled, lin1_output, lin1_output_dropouted, lin2_output, soft = your_model(batched_data, None)
                    preds.extend(soft.tolist())
        else:
            your_model.eval()
            for batched_data, edge_mask in zip(your_dataset, masked):
                reals.extend(batched_data.y.tolist())
                if your_model.__class__.__name__ == "GCN_plus_GAP":
                    self.apply_masks(your_model, edge_mask, batched_data.edge_index, apply_sigmoid=True)
                    Output_of_Hidden_Layers, pooling_layer_output, ffn_output, soft = your_model(batched_data)
                    preds.extend(soft.tolist())
                    self.clear_masks(your_model)
                elif your_model.__class__.__name__ == "DGCNN_Model":
                    final_GNN_layer_output, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, output_h1, dropout_output_h1, output_h2, soft = your_model(batched_data, edge_mask.tolist())
                    preds.extend(soft.tolist())
                elif your_model.__class__.__name__ == "DIFFPOOL_Model":
                    concatination_list_of_poolings, prediction_output_not_softed, soft = your_model(batched_data, edge_mask.tolist())
                    preds.extend(soft.tolist())
                elif your_model.__class__.__name__ == "GIN_Model":
                    mlps_output_embeds, mlp_outputs_globalSUMpooled, lin1_output, lin1_output_dropouted, lin2_output, soft = your_model(batched_data, edge_mask.tolist())
                    preds.extend(soft.tolist())


        # preds = torch.cat(preds)
        # preds, max_idxs = torch.max(preds[:], dim=1)
        # print("preds: ", preds)
        is_binary = len(np.unique(reals)) == 2
        if is_binary:
            preds = (np.array(preds)[:, 1] > 0.5).astype(int)
        roc_auc = metrics.roc_auc_score(np.array(reals), np.array(preds), multi_class="ovr", average="micro")
        return roc_auc

    def normalize_saliency_edge_based(self, saliency_maps):
        Graphs_new_gradients = []

        for graph_grads in saliency_maps:
            for edge_grads in graph_grads:
                edge_grads = (edge_grads-torch.min(graph_grads))/(torch.max(graph_grads)-torch.min(graph_grads)) if (torch.max(graph_grads)-torch.min(graph_grads)) != 0 else 0
            Graphs_new_gradients.append(graph_grads)

        return Graphs_new_gradients

    def my_fidelity(self, saliencies_for_multiple_classes, importance_threshold, style):
        if style == "Node Feature":
            try:
                fid_scores = []
                for key, value in saliencies_for_multiple_classes.items():
                    saliency_map = self.normalize_saliency_node_features_based(value)
                    fid_score = self.Fidelity_node_features(self.test_data, saliency_map, importance_threshold)
                    fid_scores.append(fid_score)

                return mean(fid_scores)
            except:
                print("attributions are not in appropriate shape")

        elif style == "Node":
            try:
                fid_scores = []
                for key, value in saliencies_for_multiple_classes.items():
                    saliency_map = self.normalize_saliency_node_based(value)
                    fid_score = self.Fidelity_node(self.test_data, saliency_map, importance_threshold)
                    fid_scores.append(fid_score)
                return mean(fid_scores)
            except:
                print("attributions are not in appropriate shape")

        elif style =="Edge":
            # try:
                fid_scores = []
                for key, value in saliencies_for_multiple_classes.items():
                    saliency_map = self.normalize_saliency_edge_based(value)
                    fid_score = self.Fidelity_edge(self.test_data, saliency_map, importance_threshold)
                    fid_scores.append(fid_score)
                return mean(fid_scores)
            # except:
            #     print("attributions are not in appropriate shape")

        else:
            print("node based not covered yet")

### **Fid-**

In [ ]:
class evalaution_of_xmethods_fidelity_minus(object):
    def __init__(self, a_trained_model, test_data):
        super(evalaution_of_xmethods_fidelity_minus, self).__init__()
        self.a_trained_model = a_trained_model
        self.test_data = test_data

    def is_salient(self, score, importance_threshold):
        if importance_threshold == score == 0:
            return True
        if importance_threshold == score == 1:
            return False
        if importance_threshold < score:
            return True
        else:
            return False

    def Compute_ROC_AUC(self, your_model, your_dataset):
        preds = []
        reals = []

        your_model.eval()
        for batched_data in your_dataset:
            if your_model.__class__.__name__ == "GCN_plus_GAP":
                Output_of_Hidden_Layers, pooling_layer_output, ffn_output, prediction = your_model(batched_data)
            elif your_model.__class__.__name__ == "DGCNN_Model":
                final_GNN_layer_output, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, output_h1, dropout_output_h1, output_h2, prediction = your_model(batched_data, None)
            elif your_model.__class__.__name__ == "DIFFPOOL_Model":
                concatination_list_of_poolings, prediction_output_not_softed, prediction = your_model(batched_data, None)
            elif your_model.__class__.__name__ == "GIN_Model":
                mlps_output_embeds, mlp_outputs_globalSUMpooled, lin1_output, lin1_output_dropouted, lin2_output, prediction = your_model(batched_data, None)
            else:
                print("Model Name is not valid.")

            preds.extend(prediction.tolist())
            reals.extend(batched_data.y.tolist())

        # for i in range(len(your_dataset)):
        #     reals.append(your_dataset[i].y.tolist())

        is_binary = len(np.unique(reals)) == 2
        if is_binary:
            preds = (np.array(preds)[:, 1] > 0.5).astype(int)

        auc_roc = metrics.roc_auc_score(np.array(reals), np.array(preds), multi_class="ovr", average="micro")

        # preds = torch.cat(preds)
        # preds, max_idxs = torch.max(preds[:], dim=1)

        # auc_roc = metrics.roc_auc_score(reals, preds, average='macro')
        return auc_roc

    def drop_unimportant_node_features(self, your_dataset, importance_threshold, attribution_scores):
        occluded_GNNgraph_list = []
        for i in range(len(attribution_scores)):
            sample_graph = deepcopy(your_dataset[i])
            for j in range(len(attribution_scores[i])):
                for k in range(len(attribution_scores[i][j])):
                    if self.is_salient(attribution_scores[i][j][k], importance_threshold) == False:  ################## change is here
                        sample_graph.x[j][k] = 0
            occluded_GNNgraph_list.append(sample_graph)
        return occluded_GNNgraph_list

    def drop_unimportant_nodes(self, your_dataset, importance_threshold, attribution_scores):
        occluded_GNNgraph_list = []
        for i in range(len(attribution_scores)):
            sample_graph = deepcopy(your_dataset[i])
            for j in range(len(attribution_scores[i])):
                if self.is_salient(attribution_scores[i][j], importance_threshold) == False:
                    sample_graph.x[j][:] = 0
            occluded_GNNgraph_list.append(sample_graph)
        return occluded_GNNgraph_list

    def Fidelity_node_features(self, your_dataset, generated_saliency_maps, importance_threshold):

        auc_roc_before_droping_unimportant_nodes = self.Compute_ROC_AUC(self.a_trained_model, your_dataset)
        new_graph_dataset = self.drop_unimportant_node_features(your_dataset, importance_threshold, generated_saliency_maps)
        auc_roc_after_droping_unimportant_nodes = self.Compute_ROC_AUC(self.a_trained_model, new_graph_dataset)

        return auc_roc_before_droping_unimportant_nodes - auc_roc_after_droping_unimportant_nodes

    def Fidelity_node(self, your_dataset, generated_saliency_maps, importance_threshold):

        auc_roc_before_droping_unimportant_nodes = self.Compute_ROC_AUC(self.a_trained_model, your_dataset)
        new_graph_dataset = self.drop_unimportant_nodes(your_dataset, importance_threshold, generated_saliency_maps)
        auc_roc_after_droping_unimportant_nodes = self.Compute_ROC_AUC(self.a_trained_model, new_graph_dataset)

        return auc_roc_before_droping_unimportant_nodes - auc_roc_after_droping_unimportant_nodes

    def drop_unimportant_edges(self, your_dataset, importance_threshold, Edge_Masks_Dropped):

        attribution_scores = []
        Edge_Masks_original = []
        Edge_Masks_Dropped_copy = []
        for tensor in Edge_Masks_Dropped:
            Edge_Masks_original.append(deepcopy(tensor.detach()))
            Edge_Masks_Dropped_copy.append(deepcopy(tensor.detach()))

        for edge_mask in Edge_Masks_Dropped_copy:

            unimportance_indices = edge_mask <= importance_threshold * (max(edge_mask)-min(edge_mask)) # Changed here > to <=
            edge_mask[unimportance_indices] = 0

        return Edge_Masks_original, Edge_Masks_Dropped_copy

    def Fidelity_edge(self, your_dataset, generated_saliency_maps, importance_threshold):

        auc_roc_before_droping_unimportant_nodes = self.Compute_ROC_AUC_edge(self.a_trained_model, your_dataset, False)
        Edge_Masks_original, Edge_Masks_Dropped = self.drop_unimportant_edges(your_dataset, importance_threshold, generated_saliency_maps)
        auc_roc_after_droping_unimportant_nodes = self.Compute_ROC_AUC_edge(self.a_trained_model, your_dataset, Edge_Masks_Dropped)

        return auc_roc_before_droping_unimportant_nodes - auc_roc_after_droping_unimportant_nodes

    def normalize_saliency_node_features_based(self, sal_maps):
        Graphs_new_gradients = []
        for graph_grads in sal_maps:
            new_gradients = []
            for node_grads in graph_grads:
                node_gradients = []
                for dim in node_grads:
                    node_gradients.append((dim-min(node_grads))/(max(node_grads)-min(node_grads)))
                new_gradients.append(node_gradients)
            Graphs_new_gradients.append(new_gradients)

        return Graphs_new_gradients

    def normalize_saliency_node_based(self, sal_maps):
        Graphs_new_gradients = []
        for graph_grads in sal_maps:
            new_gradients = []
            for node_grads in graph_grads:
                val = (node_grads-min(graph_grads))/(max(graph_grads)-min(graph_grads)) if (max(graph_grads)-min(graph_grads)) != 0 else 0
                new_gradients.append(val)
            Graphs_new_gradients.append(new_gradients)

        return Graphs_new_gradients

    def clear_masks(self, model):

        for module in model.modules():
            if isinstance(module, MessagePassing):
                module.explain = False
                module._edge_mask = None
                module._loop_mask = None
                module._apply_sigmoid = True
        return module

    def apply_masks(self, model, mask, edge_index, apply_sigmoid):
        loop_mask = edge_index[0] != edge_index[1]

        for module in model.modules():
            if isinstance(module, MessagePassing):

                if (not isinstance(mask, Parameter)
                        and '_edge_mask' in module._parameters):
                    mask = Parameter(mask)

                module.explain = True
                module._edge_mask = mask
                module._loop_mask = loop_mask
                module._apply_sigmoid = apply_sigmoid

    def Compute_ROC_AUC_edge(self, your_model, your_dataset, masked):
        preds = []
        reals = []

        if masked == False:
            your_model.eval()
            for batched_data in your_dataset:
                reals.extend(batched_data.y.tolist())
                if your_model.__class__.__name__ == "GCN_plus_GAP":
                    Output_of_Hidden_Layers, pooling_layer_output, ffn_output, soft = your_model(batched_data)
                    preds.extend(soft.tolist())
                elif your_model.__class__.__name__ == "DGCNN_Model":
                    final_GNN_layer_output, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, output_h1, dropout_output_h1, output_h2, soft = your_model(batched_data, None)
                    preds.extend(soft.tolist())
                elif your_model.__class__.__name__ == "DIFFPOOL_Model":
                    concatination_list_of_poolings, prediction_output_not_softed, soft = your_model(batched_data, None)
                    preds.extend(soft.tolist())
                elif your_model.__class__.__name__ == "GIN_Model":
                    mlps_output_embeds, mlp_outputs_globalSUMpooled, lin1_output, lin1_output_dropouted, lin2_output, soft = your_model(batched_data, None)
                    preds.extend(soft.tolist())
        else:
            your_model.eval()
            for batched_data, edge_mask in zip(your_dataset, masked):
                reals.extend(batched_data.y.tolist())
                if your_model.__class__.__name__ == "GCN_plus_GAP":
                    self.apply_masks(your_model, edge_mask, batched_data.edge_index, apply_sigmoid=True)
                    Output_of_Hidden_Layers, pooling_layer_output, ffn_output, soft = your_model(batched_data)
                    preds.extend(soft.tolist())
                    self.clear_masks(your_model)
                elif your_model.__class__.__name__ == "DGCNN_Model":
                    final_GNN_layer_output, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, output_h1, dropout_output_h1, output_h2, soft = your_model(batched_data, edge_mask.tolist())
                    preds.extend(soft.tolist())
                elif your_model.__class__.__name__ == "DIFFPOOL_Model":
                    concatination_list_of_poolings, prediction_output_not_softed, soft = your_model(batched_data, edge_mask.tolist())
                    preds.extend(soft.tolist())
                elif your_model.__class__.__name__ == "GIN_Model":
                    mlps_output_embeds, mlp_outputs_globalSUMpooled, lin1_output, lin1_output_dropouted, lin2_output, soft = your_model(batched_data, edge_mask.tolist())
                    preds.extend(soft.tolist())

        # for i, batched_graph in enumerate(your_dataset):
        #     reals.append(batched_graph.y.cpu().detach().tolist())

        # preds = torch.cat(preds)
        # preds, max_idxs = torch.max(preds[:], dim=1)

        # roc_auc = metrics.roc_auc_score(reals, preds, average='macro')
        is_binary = len(np.unique(reals)) == 2
        if is_binary:
            preds = (np.array(preds)[:, 1] > 0.5).astype(int)
        roc_auc = metrics.roc_auc_score(np.array(reals), np.array(preds), multi_class="ovr", average="micro")
        return roc_auc

    def normalize_saliency_edge_based(self, saliency_maps):
        Graphs_new_gradients = []

        for graph_grads in saliency_maps:
            for edge_grads in graph_grads:
                edge_grads = (edge_grads-torch.min(graph_grads))/(torch.max(graph_grads)-torch.min(graph_grads)) if (torch.max(graph_grads)-torch.min(graph_grads)) != 0 else 0
            Graphs_new_gradients.append(graph_grads)

        return Graphs_new_gradients

    def my_fidelity(self, saliencies_for_multiple_classes, importance_threshold, style):
        if style == "Node Feature":
            try:
                fid_scores = []
                for key, value in saliencies_for_multiple_classes.items():
                    saliency_map = self.normalize_saliency_node_features_based(value)
                    fid_score = self.Fidelity_node_features(self.test_data, saliency_map, importance_threshold)
                    fid_scores.append(fid_score)

                return mean(fid_scores)
            except:
                print("attributions are not in appropriate shape")

        elif style == "Node":
            try:
                fid_scores = []
                for key, value in saliencies_for_multiple_classes.items():
                    saliency_map = self.normalize_saliency_node_based(value)
                    fid_score = self.Fidelity_node(self.test_data, saliency_map, importance_threshold)
                    fid_scores.append(fid_score)
                return mean(fid_scores)
            except:
                print("attributions are not in appropriate shape")

        elif style =="Edge":
            try:
                fid_scores = []
                for key, value in saliencies_for_multiple_classes.items():
                    saliency_map = self.normalize_saliency_edge_based(value)
                    fid_score = self.Fidelity_edge(self.test_data, saliency_map, importance_threshold)
                    fid_scores.append(fid_score)
                return mean(fid_scores)
            except:
                print("attributions are not in appropriate shape")

        else:
            print("node based not covered yet")

### **Stability**

In [ ]:
class evalaution_of_xmethods_stability(object):
    def __init__(self, a_trained_model, test_data):
        """
            Perturb Node Features for Entire Dataset
            Explain perturbed dataset
            Compute their distance
        """
        super(evalaution_of_xmethods_stability, self).__init__()

        self.a_trained_model = a_trained_model
        self.test_data = test_data


    def normalize_saliency_node_features_based(self, sal_maps):
        Graphs_new_gradients = []
        for graph_grads in sal_maps:
            new_gradients = []
            for node_grads in graph_grads:
                node_gradients = []
                for dim in node_grads:
                    node_gradients.append((dim-min(node_grads))/(max(node_grads)-min(node_grads)) if (max(node_grads)-min(node_grads)) != 0 else 0)
                new_gradients.append(node_gradients)
            Graphs_new_gradients.append(new_gradients)

        return Graphs_new_gradients

    def normalize_saliency_node_based(self, sal_maps):
        Graphs_new_gradients = []
        for graph_grads in sal_maps:
            new_gradients = []
            for node_grads in graph_grads:
                val = (node_grads-min(graph_grads))/(max(graph_grads)-min(graph_grads)) if (max(graph_grads)-min(graph_grads)) != 0 else 0
                new_gradients.append(val)
            Graphs_new_gradients.append(new_gradients)

        return Graphs_new_gradients

    def normalize_stability_distance(self, distance_list):
        normalized_distance_list = []
        for exp_diff in distance_list:
            val = (exp_diff-min(distance_list))/(max(distance_list)-min(distance_list)) if (max(distance_list)-min(distance_list)) != 0 else 0
            normalized_distance_list.append(val)
        return normalized_distance_list

    def perturb_node_features_of_dataset(self, dataset, mean, std):
        dataset_perturbed = deepcopy(dataset)
        for graph in dataset_perturbed:
            continuous_noise = torch.ones(dataset[0].x.size()[1]).normal_(mean, std)
            graph.x += continuous_noise
        return dataset_perturbed

    def dist_explanation(self, normal_saliency, perturbed_saliency, top_k_features):
        distance_list = []
        for normal_graph, perturbed_graph in zip(normal_saliency, perturbed_saliency):
            perturbed_graph = np.array(perturbed_graph)
            perturbed_graph = torch.from_numpy(perturbed_graph)
            normal_graph = np.array(normal_graph)
            normal_graph = torch.from_numpy(normal_graph)

            normal_graph_mask = torch.where(normal_graph >= normal_graph.topk(top_k_features)[0][-1].item(),
                                            torch.ones_like(normal_graph), torch.zeros_like(normal_graph))
            perturbed_graph_mask = torch.where(perturbed_graph >= perturbed_graph.topk(top_k_features)[0][-1].item(),
                                                torch.ones_like(perturbed_graph), torch.zeros_like(perturbed_graph))

            distance = F.pairwise_distance(normal_graph_mask.unsqueeze(dim=0), perturbed_graph_mask.unsqueeze(dim=0), p=1) / normal_graph_mask.size()[0]
            distance_list.append(distance.tolist()[0])
        return distance_list

    def one_graph_all_together(self, normal_saliency, perturbed_saliency):
        normal_saliency_list = []
        perturbed_saliency_list = []
        for normal_graph, perturbed_graph in zip(normal_saliency, perturbed_saliency):
            normal_saliency_list.append([node_feat for node in normal_graph for node_feat in node])
            perturbed_saliency_list.append([node_feat for node in perturbed_graph for node_feat in node])
        return perturbed_saliency_list, perturbed_saliency_list

    def distance_thresholding_node_features(self, distance_list, importance_threshold):
        stability_list = []
        for graph_score in distance_list:
            if importance_threshold == graph_score == 0:
                stability_list.append(1)
            if importance_threshold == graph_score == 1:
                stability_list.append(0)
            if importance_threshold < graph_score:
                stability_list.append(1 - graph_score)
            else:
                stability_list.append(0)
        return stability_list

    def normalize_saliency_edge_based(self, saliency_maps):
        Graphs_new_gradients = []

        for graph_grads in saliency_maps:
            #new_gradients = []
            for edge_grads in graph_grads:
                edge_grads = (edge_grads-torch.min(graph_grads))/(torch.max(graph_grads)-torch.min(graph_grads)) if (torch.max(graph_grads)-torch.min(graph_grads)) != 0 else 0
                #new_gradients.append(val)
            Graphs_new_gradients.append(graph_grads.tolist())

        return Graphs_new_gradients

    def my_stability(self, normal_saliencies_for_multiple_classes, perturbed_saliencies_for_multiple_classes,
                     top_k_features, importance_threshold, style):
        if style == "Node Feature":
            try:
                normal_saliency_classes_listed = {}
                perturbed_saliency_classes_listed = {}
                distances_class_based = {}
                for key, values in normal_saliencies_for_multiple_classes.items():
                    normal_saliencies_for_multiple_classes[key] = self.normalize_saliency_node_features_based(normal_saliencies_for_multiple_classes[key])
                    perturbed_saliencies_for_multiple_classes[key] = self.normalize_saliency_node_features_based(perturbed_saliencies_for_multiple_classes[key])

                    normal_saliency_classes_listed[key], perturbed_saliency_classes_listed[key] = self.one_graph_all_together(normal_saliency_class0, perturbed_saliency_class0)
                    distances_class_based[key] = self.dist_explanation(normal_saliency_classes_listed[key], perturbed_saliency_classes_listed[key], top_k_features)
                    distances_class_based[key] = self.normalize_stability_distance(distances_class_based[key])
                    distances_class_based[key] = self.distance_thresholding_node_features(distances_class_based[key], importance_threshold)

                distance_list = []
                for key, value in distances_class_based.items():
                    distance_list.append(mean(value))

                return mean(distance_list)
            except:
                print("attributions are not in appropriate shape")

        elif style == "Node":
            #try:
                normal_saliency_classes_listed = {}
                perturbed_saliency_classes_listed = {}
                distances_class_based = {}
                for key, values in normal_saliencies_for_multiple_classes.items():
                    normal_saliencies_for_multiple_classes[key] = self.normalize_saliency_node_based(normal_saliencies_for_multiple_classes[key])
                    perturbed_saliencies_for_multiple_classes[key] = self.normalize_saliency_node_based(perturbed_saliencies_for_multiple_classes[key])

                    distances_class_based[key] = self.dist_explanation(normal_saliencies_for_multiple_classes[key], perturbed_saliencies_for_multiple_classes[key], top_k_features)
                    distances_class_based[key] = self.normalize_stability_distance(distances_class_based[key])
                    distances_class_based[key] = self.distance_thresholding_node_features(distances_class_based[key], importance_threshold)

                distance_list = []
                for key, value in distances_class_based.items():
                    distance_list.append(mean(value))

                return mean(distance_list)
            #except:
            #    print("attributions are not in appropriate shape")

        elif style =="Edge":
            try:
                normal_saliency_classes_listed = {}
                perturbed_saliency_classes_listed = {}
                distances_class_based = {}
                for key, values in normal_saliencies_for_multiple_classes.items():
                    normal_saliencies_for_multiple_classes[key] = self.normalize_saliency_edge_based(normal_saliencies_for_multiple_classes[key])
                    perturbed_saliencies_for_multiple_classes[key] = self.normalize_saliency_edge_based(perturbed_saliencies_for_multiple_classes[key])

                    distances_class_based[key] = self.dist_explanation(normal_saliencies_for_multiple_classes[key], perturbed_saliencies_for_multiple_classes[key], top_k_features)
                    distances_class_based[key] = self.normalize_stability_distance(distances_class_based[key])
                    distances_class_based[key] = self.distance_thresholding_node_features(distances_class_based[key], importance_threshold)

                distance_list = []
                for key, value in distances_class_based.items():
                    distance_list.append(mean(value))

                return mean(distance_list)
            except:
                print("attributions are not in appropriate shape")
        else:
            print("Only node_feat, node, and edge are covered.")

#stability = evalaution_of_xmethods_stability(GNN_Model, test_dataset)

#perturbed_dataset = stability.perturb_node_features_of_dataset(test_dataset, 0.1, 0.1)

#stability.my_stability(normal_saliency=importance_levels_for_feature_of_nodes_one_node_based,
#                       perturbed_saliency=importance_levels_for_feature_of_nodes_one_node_based_perturbed,
#                       top_k_features=10, importance_threshold=0.0000000001, style="Node")